# Dataset Preparation

In [1]:
!pip install python-terrier -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.4/347.4 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 519.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
from IPython.display import display
from xml.dom.minidom import parse, parseString
from pyterrier.measures import *

import random
import pyterrier as pt
import pandas as pd
import json
import os
import re
from tqdm import tqdm
tqdm.pandas()
from gensim.models import Word2Vec, FastText
from transformers import AutoTokenizer, TFBertModel
from scipy.spatial import distance
import tensorflow as tf
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import string
import math

if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

In [4]:
articles = pd.read_excel("/content/drive/MyDrive/Tugas Akhir/Data/df_articles.xlsx")
train_data = pd.read_excel("/content/drive/MyDrive/Tugas Akhir/Data/df_query.xlsx")

In [5]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 781 entries, 0 to 780
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        781 non-null    object
 1   docno       776 non-null    object
 2   part        781 non-null    object
 3   chap        781 non-null    object
 4   sect        695 non-null    object
 5   subsect     311 non-null    object
 6   subsubsect  763 non-null    object
dtypes: object(7)
memory usage: 42.8+ KB


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1402 entries, 0 to 1401
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   qid       1402 non-null   object
 1   query     1402 non-null   object
 2   entail    1402 non-null   object
 3   label     1402 non-null   int64 
 4   art       1402 non-null   object
 5   art_code  1402 non-null   object
dtypes: int64(1), object(5)
memory usage: 65.8+ KB


In [7]:
articles.dropna(subset=['docno'], inplace=True)
articles.fillna('', inplace=True)
articles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 776 entries, 0 to 780
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        776 non-null    object
 1   docno       776 non-null    object
 2   part        776 non-null    object
 3   chap        776 non-null    object
 4   sect        776 non-null    object
 5   subsect     776 non-null    object
 6   subsubsect  776 non-null    object
dtypes: object(7)
memory usage: 48.5+ KB


# Legal Stopword

In [8]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
articles_text = articles['text'].tolist()
query_text = train_data['query'].tolist()

combined_text = articles_text + query_text

In [14]:
def preprocess_stop(text):
  # Remove punctuation and convert to lowercase
  text = text.translate(str.maketrans('', '', string.punctuation)).lower()
  # Tokenize the text
  tokens = nltk.word_tokenize(text)
  return tokens


all_tokens = []
for sentence in combined_text:
  all_tokens.extend(preprocess_stop(sentence))
word_freq = Counter(all_tokens)

unique_word = len(word_freq)
threshold = math.ceil(0.02 * unique_word)

additional_stop = word_freq.most_common(threshold)
add_stopwords = [word[0] for word in additional_stop]

In [15]:
for word in add_stopwords:
  stop_words.add(word)

# Doc Expand

Dataset yang digunakan adalah dataset yang telah dilakukan ekspansi, ekspansi dilakukan secara terpisah karena memakan waktu yang cukup lama

In [16]:
df_beam_5 = pd.read_excel("/content/drive/MyDrive/Tugas Akhir/Data/beam_split_5_cutted_version.xlsx")
df_beam_10 = pd.read_excel("/content/drive/MyDrive/Tugas Akhir/Data/beam_split_10_cutted_version.xlsx")
df_random_5 = pd.read_excel("/content/drive/MyDrive/Tugas Akhir/Data/random_split_5_cutted_version.xlsx")
df_random_10 = pd.read_excel("/content/drive/MyDrive/Tugas Akhir/Data/random_split_5_cutted_version.xlsx")
df_beam_7 = pd.read_excel("/content/drive/MyDrive/Tugas Akhir/Data/beam_split_7_cutted_version.xlsx")
df_random_7 = pd.read_excel("/content/drive/MyDrive/Tugas Akhir/Data/random_split_7_cutted_version.xlsx")

In [17]:
df_beam_5.dropna(subset=['docno'], inplace=True)
df_beam_5.fillna('', inplace=True)
df_beam_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   docno       776 non-null    object
 1   text        776 non-null    object
 2   part        776 non-null    object
 3   chap        776 non-null    object
 4   sect        776 non-null    object
 5   subsect     776 non-null    object
 6   subsubsect  776 non-null    object
dtypes: object(7)
memory usage: 42.6+ KB


In [18]:
df_beam_10.dropna(subset=['docno'], inplace=True)
df_beam_10.fillna('', inplace=True)
df_beam_10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   docno       776 non-null    object
 1   text        776 non-null    object
 2   part        776 non-null    object
 3   chap        776 non-null    object
 4   sect        776 non-null    object
 5   subsect     776 non-null    object
 6   subsubsect  776 non-null    object
dtypes: object(7)
memory usage: 42.6+ KB


In [19]:
df_random_5.dropna(subset=['docno'], inplace=True)
df_random_5.fillna('', inplace=True)
df_random_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   docno       776 non-null    object
 1   text        776 non-null    object
 2   part        776 non-null    object
 3   chap        776 non-null    object
 4   sect        776 non-null    object
 5   subsect     776 non-null    object
 6   subsubsect  776 non-null    object
dtypes: object(7)
memory usage: 42.6+ KB


In [20]:
df_random_10.dropna(subset=['docno'], inplace=True)
df_random_10.fillna('', inplace=True)
df_random_10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   docno       776 non-null    object
 1   text        776 non-null    object
 2   part        776 non-null    object
 3   chap        776 non-null    object
 4   sect        776 non-null    object
 5   subsect     776 non-null    object
 6   subsubsect  776 non-null    object
dtypes: object(7)
memory usage: 42.6+ KB


In [21]:
df_random_7.dropna(subset=['docno'], inplace=True)
df_random_7.fillna('', inplace=True)
df_random_7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   docno       776 non-null    object
 1   text        776 non-null    object
 2   part        776 non-null    object
 3   chap        776 non-null    object
 4   sect        776 non-null    object
 5   subsect     776 non-null    object
 6   subsubsect  776 non-null    object
dtypes: object(7)
memory usage: 42.6+ KB


In [22]:
df_beam_7.dropna(subset=['docno'], inplace=True)
df_beam_7.fillna('', inplace=True)
df_beam_7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776 entries, 0 to 775
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   docno       776 non-null    object
 1   text        776 non-null    object
 2   part        776 non-null    object
 3   chap        776 non-null    object
 4   sect        776 non-null    object
 5   subsect     776 non-null    object
 6   subsubsect  776 non-null    object
dtypes: object(7)
memory usage: 42.6+ KB


# Word Embedding

## Query Expansion W2V



### Data Preprocessing

In [23]:
seed = 111

In [24]:
# Function to transform sentence in DataFrame to a list of words
def sentence_to_list(sentence):
    return sentence.lower().split()
low = pd.DataFrame()
# Apply the function to the 'Sentences' column
low['sentences'] = articles['text'].apply(sentence_to_list)
low

,sentences
0,"[article, 1, 1, private, rights, must, be, con..."
1,"[article, 2, this, code, must, be, construed, ..."
2,"[article, 3, 1, the, enjoyment, of, private, r..."
3,"[article, 3, 2, if, the, person, making, a, ju..."
4,"[article, 4, the, age, of, majority, is, 20, y..."
...,...
776,"[article, 721, an, unborn, child, is, deemed, ..."
777,"[article, 722, 1, the, provisions, of, article..."
778,"[article, 723, the, court, may, order, a, pers..."
779,"[article, 724, in, the, following, cases, the,..."


In [25]:
q=pd.DataFrame()
q['sentences'] = train_data['query'].apply(sentence_to_list)
q

,sentences
0,"[in, the, case, where, a, person, under, curat..."
1,"[in, cases, any, third, party, commits, any, f..."
2,"[in, the, case, where, a, person, under, curat..."
3,"[in, the, case, where, a, person, under, curat..."
4,"[a, person, who, intends, to, conclude, the, c..."
...,...
1397,"[in, a, lawsuit, demanding, the, payment, of, ..."
1398,"[the, beneficiary, in, bad, faith, of, unjust,..."
1399,"[even, if, an, obligation, does, not, exist, a..."
1400,"[in, cases, where, an, obligor, mistakenly, be..."


In [26]:
df_training = pd.concat([low, q],ignore_index=True)
# low.append(q,)
final_data = df_training['sentences'].tolist()
len(final_data)

2178

In [27]:
final_data[2]

['article',
 '3',
 '1',
 'the',
 'enjoyment',
 'of',
 'private',
 'rights',
 'commences',
 'at',
 'birth',
 '2',
 'unless',
 'otherwise',
 'prohibited',
 'by',
 'applicable',
 'laws',
 'regulations',
 'or',
 'treaties',
 'foreign',
 'nationals',
 'enjoy',
 'private',
 'rights']

### Word2Vec Model

In [28]:


model_sg = Word2Vec(window=4, min_count=1,  workers=4, vector_size=128, sg=1, seed=seed)
model_sg.build_vocab(final_data, progress_per=1000)

model_cbow = Word2Vec(window=4, min_count=1,  workers=4 , vector_size=128, seed=seed)
model_cbow.build_vocab(final_data, progress_per=1000)

In [29]:
model_sg.train(final_data, total_examples=model_sg.corpus_count, epochs=100)
model_cbow.train(final_data, total_examples=model_cbow.corpus_count, epochs=100)

(7523573, 11619900)

In [30]:
model_sg.save("skipgram.w2v")
model_cbow.save("cbow.w2v")

In [31]:
model_sg = Word2Vec.load("skipgram.w2v")
model_cbow = Word2Vec.load("cbow.w2v")

In [32]:
w2v_sg= model_sg.wv
w2v_cbow = model_cbow.wv

## Fast Text Ekspansion

### Model

In [33]:
model_ft_sg = FastText(window=4, min_count=1,  workers=4, vector_size=128, sg=1, seed=seed)
model_ft_sg.build_vocab(final_data, progress_per=1000)
model_ft_sg.train(final_data, total_examples=model_sg.corpus_count, epochs=100)

model_ft_cb = FastText(window=4, min_count=1, vector_size=128, workers=4, sg=0, seed=seed)
model_ft_cb.build_vocab(final_data, progress_per=1000)
model_ft_cb.train(final_data, total_examples=model_sg.corpus_count, epochs=100)

(7524457, 11619900)

In [34]:
model_ft_sg.save("fasttext_sg.fasttext")
model_ft_cb.save("fasttext_cb.fasttext")

In [35]:
model_ft_sg = FastText.load("fasttext_sg.fasttext")
model_ft_cb = FastText.load("fasttext_cb.fasttext")

In [36]:
fasttext_sg = model_ft_sg.wv
fasttext_cb = model_ft_cb.wv

## Bert Contextual Similarity Check

### Load Pre-Trained Model

In [37]:


bert_tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased', use_fast=True, max_length=512)
bert_model = TFBertModel.from_pretrained('google-bert/bert-base-uncased', output_hidden_states=True, from_pt = True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

In [38]:
def tokenized_texts(text):

  return bert_tokenizer(text, return_tensors='pt', truncation=True)

def word_pos(text):
  list_of_word_pos = []
  tokenized_text = bert_tokenizer.tokenize(text, add_special_tokens=True)
  i = 0
  while i < len(tokenized_text) - 1:
    curr_token, next_token = tokenized_text[i], tokenized_text[i + 1]
    if next_token.startswith("##"):
      whole_word = curr_token
      pos = [i]
      i += 1
      while i < len(tokenized_text) and tokenized_text[i].startswith("##"):
        whole_word += tokenized_text[i][2:]
        pos.append(i)
        i += 1
      list_of_word_pos.append((whole_word, pos))
    else:
      list_of_word_pos.append((curr_token, [i]))
      i += 1
  return list_of_word_pos

In [39]:


def get_token_vecs(token_embeddings):
  token_vecs_sum = []
  for token in token_embeddings:
    sum_vec = tf.reduce_sum(token[-4:], axis = 0)
    token_vecs_sum.append(sum_vec)
  return token_vecs_sum

def similarity_vector(encoded_input):
  token_ids = np.array(encoded_input["input_ids"])
  token_types = np.array(encoded_input["token_type_ids"])
  token_masks = np.array(encoded_input["attention_mask"])

  outputs = bert_model([token_ids, token_types, token_masks])
  hidden_states = outputs[2]
  stacked_hidden_states = tf.stack(hidden_states)

  token_embeddings_sentence = tf.transpose(stacked_hidden_states, perm=[1, 2, 0, 3])

  return get_token_vecs(token_embeddings_sentence[0])

def cosine_sim(a, b):
  return 1 - distance.cosine(a, b)


In [40]:
def process(sentence, th):

  tokenized_text = tokenized_texts(sentence)
  word_position = word_pos(sentence)
  token = similarity_vector(tokenized_text)
  mean = tf.reduce_mean(token[word_position[0][1][0] : word_position[-1][1][-1]], axis=0)
  lst = []

  for i in word_position:
    if len(i[1]) == 1:
      sim = cosine_sim(mean, token[i[1][0]])
      if sim > th :
        lst.append(i[0])
    else:
      temp = tf.reduce_mean(token[i[1][0] :i[1][-1]+1], axis=0)
      sim = cosine_sim(mean, temp)
      if sim > th:
        lst.append(i[0])

  return lst

# Query Expansion

In [41]:
train_data_1 = train_data.copy()

In [42]:
def get_similar(model, word, threshold):
  data = model.most_similar(word, topn=1)
  result = [entry[0] for entry in data if entry[1]> threshold]
  return result

In [43]:


# nltk.download('stopwords')
# stop_words = set(stopwords.words('english'))
# stop_words = set()
def expand_query(query, model, threshold,th, isContext):
  res = []
  res_sentence=""
  q = query['query']
  if isContext :
    sentence = process(q, th)
  else:
    sentence = q.split()

  for word in sentence:
    if word not in stop_words:
      try:
        hasil = get_similar(model, word.lower(), threshold)
      except KeyError:
        continue
      else:
        for i in hasil:
          if i in res:
            continue
          else:
            res.append(" "+i)
        continue

  for i in res:
    res_sentence += i

  return q + res_sentence

In [44]:
threshold = 0.5
bert_th = 0.5

# Retrieval

## Indexing

In [45]:
!rm -rf ./coliee_index

# we need to use blocks = True
# All indexer classes expose a blocks boolean constructor argument to allow position information to be
# recoreded in the index. Defaults to False, i.e. positions are not recorded.
# Indexing will record "position" aka blocks information, however this makes the indices much larger and also slower.
pd_indexer = pt.DFIndexer("./coliee_index", type=pt.index.IndexingType(1), blocks = True)

# note that we also keep the articles as "text" in the metadata part
# (Add the entire dataframe as metadata)
# this will be useful when we re-rank model using LLM, such as T5, BERT, etc.
index_ref = pd_indexer.index(articles["text"], \
                             articles["text"], \
                             articles["docno"], \
                             articles["part"], \
                             articles["chap"], \
                             articles["sect"], \
                             articles["subsect"], \
                             articles["subsubsect"])

In [46]:
!rm -rf ./coliee_beam_5_index
pd_indexerb5 = pt.DFIndexer("./coliee_beam_5_index", type=pt.index.IndexingType(1), blocks = True)
index_ref_beam5 = pd_indexerb5.index(df_beam_5["text"], df_beam_5)

!rm -rf ./coliee_beam_10_index
pd_indexerb10 = pt.DFIndexer("./coliee_beam_10_index", type=pt.index.IndexingType(1), blocks = True)
index_ref_beam10 = pd_indexerb10.index(df_beam_10["text"], df_beam_10)

!rm -rf ./coliee_beam_7_index
pd_indexerb7 = pt.DFIndexer("./coliee_beam_7_index", type=pt.index.IndexingType(1), blocks = True)
index_ref_beam7 = pd_indexerb7.index(df_beam_7["text"], df_beam_7)

!rm -rf ./coliee_random_5_index
pd_indexerr5 = pt.DFIndexer("./coliee_random_5_index", type=pt.index.IndexingType(1), blocks = True)
index_ref_rand5 = pd_indexerr5.index(df_random_5["text"], df_random_5)

!rm -rf ./coliee_random_10_index
pd_indexerr10 = pt.DFIndexer("./coliee_random_10_index", type=pt.index.IndexingType(1), blocks = True)
index_ref_rand10 = pd_indexerr10.index(df_random_10["text"], df_random_10)

!rm -rf ./coliee_random_7_index
pd_indexerr7 = pt.DFIndexer("./coliee_random_7_index", type=pt.index.IndexingType(1), blocks = True)
index_ref_rand7 = pd_indexerr7.index(df_random_7["text"], df_random_7)

## Global Query Expansion Pipeline

Embedding Model Available:
*   w2v_sg
*   w2v_cbow
*   fasttext_sg
*   fasttext_cbow

Hyperparameter:
*   Contextual
    *   Threshold 0.6
    *   Threshold 0.5
*   Embedding Model Threshold
    *   Threshold 0.4
    *   Threshold 0.45    






In [47]:
# experiment

# topics --- at the end we only run experiment for R04 (test data for COLIEE 2023)
topics = train_data[["qid", "query"]].drop_duplicates()
topics['category'] = topics.apply(lambda row: row.qid.split('-')[0], axis = 1)
topics_R04 = topics[topics["category"] == 'R04']

# creating qrels ---> all articles in the training data are relevant; all others
# are assumed to be non-relevant (including negative samples)
qrels_temp = train_data[["qid", "art_code", "label"]].copy()
qrels_q = qrels_temp.rename(columns={"art_code": "docno"})

### BM25 Model

In [48]:
bm25 = pt.BatchRetrieve(index_ref, wmodel = "BM25") % 30

# Non-Contextualized
# bm25v11 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref, wmodel = "BM25") % 30
# bm25v12 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref, wmodel = "BM25") % 30
# bm25v21 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref, wmodel = "BM25") % 30
# bm25v22 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref, wmodel = "BM25") % 30
# bm25v31 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref, wmodel = "BM25") % 30
# bm25v32 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref, wmodel = "BM25") % 30
# bm25v41 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref, wmodel = "BM25") % 30
# bm25v42 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref, wmodel = "BM25") % 30
# # Contextualized
# bm25v13 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref, wmodel = "BM25") % 30
# bm25v14 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref, wmodel = "BM25") % 30
# bm25v23 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref, wmodel = "BM25") % 30
# bm25v24 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref, wmodel = "BM25") % 30
# bm25v33 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref, wmodel = "BM25") % 30
# bm25v34 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref, wmodel = "BM25") % 30
# bm25v43 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref, wmodel = "BM25") % 30
# bm25v44 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref, wmodel = "BM25") % 30

### TF-IDF Model

In [49]:
tfidf = pt.BatchRetrieve(index_ref, wmodel = "TF_IDF") % 30

# Non-Contextualized
# tfidfv11 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref, wmodel = "TF_IDF") % 30
# tfidfv12 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref, wmodel = "TF_IDF") % 30
# tfidfv21 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref, wmodel = "TF_IDF") % 30
# tfidfv22 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref, wmodel = "TF_IDF") % 30
# tfidfv31 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref, wmodel = "TF_IDF") % 30
# tfidfv32 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref, wmodel = "TF_IDF") % 30
# tfidfv41 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref, wmodel = "TF_IDF") % 30
# tfidfv42 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref, wmodel = "TF_IDF") % 30
# # Contextualized
# tfidfv13 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref, wmodel = "TF_IDF") % 30
# tfidfv14 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref, wmodel = "TF_IDF") % 30
# tfidfv23 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref, wmodel = "TF_IDF") % 30
# tfidfv24 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref, wmodel = "TF_IDF") % 30
# tfidfv33 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref, wmodel = "TF_IDF") % 30
# tfidfv34 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref, wmodel = "TF_IDF") % 30
# tfidfv43 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref, wmodel = "TF_IDF") % 30
# tfidfv44 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref, wmodel = "TF_IDF") % 30

## Local Query Expansion Pipeline

### Pipeline

## Experiment

### Doc+Query Expansion

#### BM25 Beam 5

In [61]:
bm25v10 = pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
# Non-Contextualized

bm25v11 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v12 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v21 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v22 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v31 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v32 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v41 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v42 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30

# Contextualized
bm25v13 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v14 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v15 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v16 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30

bm25v23 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v24 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v25 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v26 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30

bm25v33 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v34 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v35 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v36 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30

bm25v43 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v44 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v45 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30
bm25v46 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30

In [62]:
for i in range(1,5):
  for j in range(1,3):
    print("bm25v"+str(i)+str(j)+',')
model_embed = ["W2V-SG", "W2V-CBOW", "FT-SG", "FT-CBOW"]



bm25v11,
bm25v12,
bm25v21,
bm25v22,
bm25v31,
bm25v32,
bm25v41,
bm25v42,


In [63]:
model_tm = "BM25"
names= [model_tm]
for a in model_embed:
  for b in range(1,3):
    names.append(model_tm+'-Rand10-Con'+str(a)+str(b)+'')
names

['BM25',
 'BM25-Rand10-ConW2V-SG1',
 'BM25-Rand10-ConW2V-SG2',
 'BM25-Rand10-ConW2V-CBOW1',
 'BM25-Rand10-ConW2V-CBOW2',
 'BM25-Rand10-ConFT-SG1',
 'BM25-Rand10-ConFT-SG2',
 'BM25-Rand10-ConFT-CBOW1',
 'BM25-Rand10-ConFT-CBOW2']

In [64]:
pt.Experiment(
    [
bm25v10,
bm25v11,
bm25v12,
bm25v21,
bm25v22,
bm25v31,
bm25v32,
bm25v41,
bm25v42,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = ['BM25',
 'BEAM5-W2V-SG1',
 'BEAM5-W2V-SG2',
 'BEAM5-W2V-CBOW1',
 'BEAM5-W2V-CBOW2',
 'BEAM5-FT-SG1',
 'BEAM5-FT-SG2',
 'BEAM5-FT-CBOW1',
 'BEAM5-FT-CBOW2'],
    verbose = True,
    baseline=0
)

pt.Experiment:   0%|          | 0/9 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,BM25,0.742088,0.186139,0.861386,0.876238,0.900990,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BEAM5-W2V-SG1,0.693947,0.180198,0.831683,0.886139,0.910891,9.0,20.0,0.038795,2.0,...,0.258873,2.0,6.0,0.181012,4.0,3.0,0.529762,3.0,1.0,0.319724
2,BEAM5-W2V-SG2,0.693976,0.180198,0.831683,0.886139,0.910891,9.0,20.0,0.038911,2.0,...,0.258873,2.0,6.0,0.181012,4.0,3.0,0.529762,3.0,1.0,0.319724
3,BEAM5-W2V-CBOW1,0.681417,0.178218,0.801980,0.851485,0.896040,8.0,24.0,0.011468,2.0,...,0.158330,2.0,8.0,0.022584,2.0,5.0,0.166650,3.0,2.0,0.764686
4,BEAM5-W2V-CBOW2,0.685744,0.176238,0.821782,0.881188,0.910891,8.0,24.0,0.015667,1.0,...,0.058402,2.0,6.0,0.073477,2.0,2.0,0.707450,2.0,0.0,0.158330
5,BEAM5-FT-SG1,0.709357,0.178218,0.851485,0.876238,0.915842,6.0,19.0,0.058357,1.0,...,0.102744,1.0,3.0,0.319724,3.0,3.0,1.000000,3.0,0.0,0.083249
6,BEAM5-FT-SG2,0.709357,0.178218,0.851485,0.876238,0.915842,6.0,19.0,0.058357,1.0,...,0.102744,1.0,3.0,0.319724,3.0,3.0,1.000000,3.0,0.0,0.083249
7,BEAM5-FT-CBOW1,0.727312,0.196040,0.851485,0.871287,0.905941,11.0,15.0,0.394087,6.0,...,0.058402,2.0,4.0,0.416930,2.0,3.0,0.656968,3.0,2.0,0.656968
8,BEAM5-FT-CBOW2,0.727312,0.196040,0.851485,0.871287,0.905941,11.0,15.0,0.394087,6.0,...,0.058402,2.0,4.0,0.416930,2.0,3.0,0.656968,3.0,2.0,0.656968


In [65]:
pt.Experiment(
    [
bm25v10,
bm25v13,
bm25v14,
bm25v15,
bm25v16,
bm25v23,
bm25v24,
bm25v25,
bm25v26,
bm25v33,
bm25v34,
bm25v35,
bm25v36,
bm25v43,
bm25v44,
bm25v45,
bm25v46,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = ['BM25', 'BEAM5-ConW2V-SG1', 'BEAM5-ConW2V-SG2','BEAM5-ConW2V-SG3', 'BEAM5-ConW2V-SG4','BEAM5-ConW2V-CBOW1','BEAM5-ConW2V-CBOW2','BEAM5-ConW2V-CBOW3','BEAM5-ConW2V-CBOW4','BEAM5-ConFT-SG1', 'BEAM5-ConFT-SG2','BEAM5-ConFT-SG3', 'BEAM5-ConFT-SG4','BEAM5-ConFT-CBOW1','BEAM5-ConFT-CBOW2','BEAM5-ConFT-CBOW3','BEAM5-ConFT-CBOW4',],
    verbose = True,
    baseline=0
)

pt.Experiment:   0%|          | 0/17 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,BM25,0.742088,0.186139,0.861386,0.876238,0.900990,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BEAM5-ConW2V-SG1,0.712292,0.182178,0.831683,0.881188,0.910891,9.0,16.0,0.153332,2.0,...,0.416930,2.0,6.0,0.181012,3.0,3.0,0.740688,3.0,1.0,0.319724
2,BEAM5-ConW2V-SG2,0.723114,0.186139,0.841584,0.881188,0.910891,10.0,13.0,0.342860,2.0,...,1.000000,1.0,4.0,0.287281,2.0,2.0,0.707450,2.0,0.0,0.158330
3,BEAM5-ConW2V-SG3,0.712321,0.182178,0.831683,0.881188,0.910891,9.0,16.0,0.153726,2.0,...,0.416930,2.0,6.0,0.181012,3.0,3.0,0.740688,3.0,1.0,0.319724
4,BEAM5-ConW2V-SG4,0.723114,0.186139,0.841584,0.881188,0.910891,10.0,13.0,0.342860,2.0,...,1.000000,1.0,4.0,0.287281,2.0,2.0,0.707450,2.0,0.0,0.158330
5,BEAM5-ConW2V-CBOW1,0.688465,0.178218,0.811881,0.861386,0.896040,8.0,23.0,0.020289,2.0,...,0.158330,2.0,7.0,0.040616,3.0,5.0,0.469595,3.0,2.0,0.764686
6,BEAM5-ConW2V-CBOW2,0.699617,0.178218,0.816832,0.881188,0.896040,9.0,19.0,0.057264,1.0,...,0.102744,1.0,6.0,0.038310,3.0,2.0,0.764686,3.0,2.0,0.764686
7,BEAM5-ConW2V-CBOW3,0.695723,0.178218,0.831683,0.881188,0.910891,8.0,21.0,0.035321,1.0,...,0.102744,2.0,5.0,0.134334,2.0,2.0,0.707450,2.0,0.0,0.158330
8,BEAM5-ConW2V-CBOW4,0.706854,0.182178,0.831683,0.891089,0.905941,10.0,17.0,0.094524,1.0,...,0.319724,1.0,4.0,0.109176,3.0,1.0,0.258873,1.0,0.0,0.319724
9,BEAM5-ConFT-SG1,0.720798,0.178218,0.856436,0.871287,0.915842,6.0,16.0,0.230960,1.0,...,0.102744,2.0,3.0,0.656968,2.0,3.0,0.656968,3.0,0.0,0.083249


#### TF-IDF Beam 5

In [66]:
tfidfv10 = pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
# Non-Contextualized
tfidfv11 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
tfidfv12 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
tfidfv21 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
tfidfv22 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30

tfidfv31 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
tfidfv32 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
tfidfv41 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
tfidfv42 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30

# Contextualized
tfidfv13 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
tfidfv14 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
tfidfv15 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
tfidfv16 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30

tfidfv23 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
tfidfv24 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
tfidfv25 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
tfidfv26 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30

tfidfv33 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
tfidfv34 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
tfidfv35 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
tfidfv36 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30

tfidfv43 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
tfidfv44 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
tfidfv45 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
tfidfv46 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30

In [67]:
pt.Experiment(
    [
tfidfv10,
tfidfv11,
tfidfv12,
tfidfv21,
tfidfv22,
tfidfv31,
tfidfv32,
tfidfv41,
tfidfv42,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = ['TF-IDF',
 'TF-IDF-BEAM5-NoConW2V-SG1',
 'TF-IDF-BEAM5-NoConW2V-SG2',
 'TF-IDF-BEAM5-NoConW2V-CBOW1',
 'TF-IDF-BEAM5-NoConW2V-CBOW2',
 'TF-IDF-BEAM5-NoConFT-SG1',
 'TF-IDF-BEAM5-NoConFT-SG2',
 'TF-IDF-BEAM5-NoConFT-CBOW1',
 'TF-IDF-BEAM5-NoConFT-CBOW2'],
    verbose = True,
    baseline=0
)

pt.Experiment:   0%|          | 0/9 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,TF-IDF,0.741401,0.190099,0.856436,0.876238,0.891089,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TF-IDF-BEAM5-NoConW2V-SG1,0.693843,0.182178,0.826733,0.881188,0.915842,8.0,20.0,0.043389,0.0,...,0.044941,3.0,7.0,0.202355,5.0,5.0,0.783057,4.0,0.0,0.058402
2,TF-IDF-BEAM5-NoConW2V-SG2,0.693843,0.182178,0.826733,0.881188,0.915842,8.0,20.0,0.043389,0.0,...,0.044941,3.0,7.0,0.202355,5.0,5.0,0.783057,4.0,0.0,0.058402
3,TF-IDF-BEAM5-NoConW2V-CBOW1,0.674321,0.174257,0.821782,0.866337,0.896040,7.0,27.0,0.005382,1.0,...,0.010704,3.0,6.0,0.127253,4.0,5.0,0.639676,3.0,1.0,0.707450
4,TF-IDF-BEAM5-NoConW2V-CBOW2,0.678035,0.176238,0.816832,0.866337,0.910891,7.0,25.0,0.005022,1.0,...,0.018867,3.0,7.0,0.088139,2.0,3.0,0.482235,3.0,0.0,0.102744
5,TF-IDF-BEAM5-NoConFT-SG1,0.717994,0.178218,0.846535,0.866337,0.900990,6.0,16.0,0.160508,0.0,...,0.013564,0.0,1.0,0.319724,1.0,3.0,0.319724,3.0,1.0,0.319724
6,TF-IDF-BEAM5-NoConFT-SG2,0.713044,0.178218,0.846535,0.866337,0.900990,6.0,17.0,0.102595,0.0,...,0.013564,0.0,1.0,0.319724,1.0,3.0,0.319724,3.0,1.0,0.319724
7,TF-IDF-BEAM5-NoConFT-CBOW1,0.724615,0.196040,0.856436,0.876238,0.886139,13.0,14.0,0.289948,5.0,...,0.258873,2.0,2.0,1.000000,2.0,3.0,1.000000,2.0,3.0,0.656968
8,TF-IDF-BEAM5-NoConFT-CBOW2,0.724615,0.196040,0.856436,0.876238,0.886139,13.0,14.0,0.289948,5.0,...,0.258873,2.0,2.0,1.000000,2.0,3.0,1.000000,2.0,3.0,0.656968


In [68]:
pt.Experiment(
    [
tfidfv10,
tfidfv13,
tfidfv14,
tfidfv15,
tfidfv16,
tfidfv23,
tfidfv24,
tfidfv25,
tfidfv26,
tfidfv33,
tfidfv34,
tfidfv35,
tfidfv36,
tfidfv43,
tfidfv44,
tfidfv45,
tfidfv46,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = ['TF-IDF',
 'TF-IDF-BEAM5-ConW2V-SG1',
 'TF-IDF-BEAM5-ConW2V-SG2',
 'TF-IDF-BEAM5-ConW2V-SG3',
 'TF-IDF-BEAM5-ConW2V-SG4',
 'TF-IDF-BEAM5-ConW2V-CBOW1',
 'TF-IDF-BEAM5-ConW2V-CBOW2',
 'TF-IDF-BEAM5-ConW2V-CBOW3',
 'TF-IDF-BEAM5-ConW2V-CBOW4',
 'TF-IDF-BEAM5-ConFT-SG1',
 'TF-IDF-BEAM5-ConFT-SG2',
 'TF-IDF-BEAM5-ConFT-SG3',
 'TF-IDF-BEAM5-ConFT-SG4',
 'TF-IDF-BEAM5-ConFT-CBOW1',
 'TF-IDF-BEAM5-ConFT-CBOW2',
 'TF-IDF-BEAM5-ConFT-CBOW3',
 'TF-IDF-BEAM5-ConFT-CBOW4'
 ],
    verbose = True,
    baseline=0
)

pt.Experiment:   0%|          | 0/17 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,TF-IDF,0.741401,0.190099,0.856436,0.876238,0.891089,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TF-IDF-BEAM5-ConW2V-SG1,0.711453,0.182178,0.831683,0.876238,0.915842,8.0,16.0,0.160817,0.0,...,0.044941,3.0,6.0,0.277400,4.0,5.0,1.000000,4.0,0.0,0.058402
2,TF-IDF-BEAM5-ConW2V-SG2,0.723919,0.188119,0.831683,0.871287,0.910891,8.0,12.0,0.379647,0.0,...,0.319724,2.0,5.0,0.253340,3.0,4.0,0.783057,3.0,0.0,0.102744
3,TF-IDF-BEAM5-ConW2V-SG3,0.711453,0.182178,0.831683,0.876238,0.915842,8.0,16.0,0.160817,0.0,...,0.044941,3.0,6.0,0.277400,4.0,5.0,1.000000,4.0,0.0,0.058402
4,TF-IDF-BEAM5-ConW2V-SG4,0.723919,0.188119,0.831683,0.871287,0.910891,8.0,12.0,0.379647,0.0,...,0.319724,2.0,5.0,0.253340,3.0,4.0,0.783057,3.0,0.0,0.102744
5,TF-IDF-BEAM5-ConW2V-CBOW1,0.685223,0.174257,0.821782,0.866337,0.896040,7.0,25.0,0.013508,1.0,...,0.010704,3.0,6.0,0.127253,4.0,5.0,0.639676,3.0,1.0,0.707450
6,TF-IDF-BEAM5-ConW2V-CBOW2,0.696728,0.178218,0.816832,0.871287,0.896040,7.0,21.0,0.043042,1.0,...,0.033206,2.0,6.0,0.073477,2.0,3.0,0.764686,3.0,1.0,0.707450
7,TF-IDF-BEAM5-ConW2V-CBOW3,0.689721,0.176238,0.826733,0.866337,0.910891,7.0,23.0,0.013987,1.0,...,0.018867,3.0,6.0,0.158330,2.0,3.0,0.482235,3.0,0.0,0.102744
8,TF-IDF-BEAM5-ConW2V-CBOW4,0.699371,0.184158,0.826733,0.881188,0.905941,7.0,19.0,0.039054,1.0,...,0.181012,2.0,5.0,0.134334,1.0,0.0,0.319724,2.0,0.0,0.181012
9,TF-IDF-BEAM5-ConFT-SG1,0.727024,0.178218,0.851485,0.871287,0.896040,7.0,15.0,0.409673,0.0,...,0.013564,1.0,1.0,0.656968,2.0,3.0,0.656968,2.0,1.0,0.566290


#### BM25 Beam 10

In [69]:
# Non-Contextualized
bm25v11 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v12 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v21 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v22 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v31 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v32 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v41 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v42 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30

# Contextualized
bm25v13 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v14 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v15 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v16 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30

bm25v23 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v24 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v25 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v26 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30

bm25v33 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v34 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v35 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v36 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30

bm25v43 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v44 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v45 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30
bm25v46 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30

In [70]:
pt.Experiment(
    [
bm25v10,
bm25v11,
bm25v12,
bm25v21,
bm25v22,
bm25v31,
bm25v32,
bm25v41,
bm25v42,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = ['BM25',
 'BM25-BEAM10-NoConW2V-SG1',
 'BM25-BEAM10-NoConW2V-SG2',
 'BM25-BEAM10-NoConW2V-CBOW1',
 'BM25-BEAM10-NoConW2V-CBOW2',
 'BM25-BEAM10-NoConFT-SG1',
 'BM25-BEAM10-NoConFT-SG2',
 'BM25-BEAM10-NoConFT-CBOW1',
 'BM25-BEAM10-NoConFT-CBOW2'],
    verbose = True,
    baseline=0
)

pt.Experiment:   0%|          | 0/9 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,BM25,0.742088,0.186139,0.861386,0.876238,0.900990,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BM25-BEAM10-NoConW2V-SG1,0.689262,0.180198,0.836634,0.900990,0.910891,8.0,20.0,0.019011,1.0,...,0.181012,3.0,6.0,0.277400,6.0,3.0,0.198192,3.0,1.0,0.319724
2,BM25-BEAM10-NoConW2V-SG2,0.689282,0.180198,0.836634,0.900990,0.910891,8.0,20.0,0.019054,1.0,...,0.181012,3.0,6.0,0.277400,6.0,3.0,0.198192,3.0,1.0,0.319724
3,BM25-BEAM10-NoConW2V-CBOW1,0.672388,0.178218,0.811881,0.856436,0.891089,6.0,26.0,0.003920,2.0,...,0.158330,2.0,7.0,0.040616,3.0,5.0,0.287281,3.0,3.0,0.566290
4,BM25-BEAM10-NoConW2V-CBOW2,0.681104,0.172277,0.816832,0.861386,0.896040,7.0,22.0,0.007063,0.0,...,0.007509,2.0,7.0,0.049027,2.0,4.0,0.319724,2.0,2.0,0.707450
5,BM25-BEAM10-NoConFT-SG1,0.712248,0.176238,0.846535,0.881188,0.905941,7.0,19.0,0.085507,1.0,...,0.058402,1.0,3.0,0.258873,4.0,3.0,0.707450,3.0,1.0,0.707450
6,BM25-BEAM10-NoConFT-SG2,0.712248,0.176238,0.846535,0.881188,0.905941,7.0,19.0,0.085507,1.0,...,0.058402,1.0,3.0,0.258873,4.0,3.0,0.707450,3.0,1.0,0.707450
7,BM25-BEAM10-NoConFT-CBOW1,0.743371,0.192079,0.856436,0.881188,0.900990,16.0,13.0,0.942580,5.0,...,0.258873,3.0,4.0,0.707450,2.0,1.0,0.566290,3.0,2.0,1.000000
8,BM25-BEAM10-NoConFT-CBOW2,0.743371,0.192079,0.856436,0.881188,0.900990,16.0,13.0,0.942580,5.0,...,0.258873,3.0,4.0,0.707450,2.0,1.0,0.566290,3.0,2.0,1.000000


In [71]:
pt.Experiment(
    [
bm25v10,
bm25v13,
bm25v14,
bm25v15,
bm25v16,
bm25v23,
bm25v24,
bm25v25,
bm25v26,
bm25v33,
bm25v34,
bm25v35,
bm25v36,
bm25v43,
bm25v44,
bm25v45,
bm25v46,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = ['BM25',
 'BM25-BEAM10-ConW2V-SG1',
 'BM25-BEAM10-ConW2V-SG2',
 'BM25-BEAM10-ConW2V-SG3',
 'BM25-BEAM10-ConW2V-SG4',
 'BM25-BEAM10-ConW2V-CBOW1',
 'BM25-BEAM10-ConW2V-CBOW2',
 'BM25-BEAM10-ConW2V-CBOW3',
 'BM25-BEAM10-ConW2V-CBOW4',
 'BM25-BEAM10-ConFT-SG1',
 'BM25-BEAM10-ConFT-SG2',
 'BM25-BEAM10-ConFT-SG3',
 'BM25-BEAM10-ConFT-SG4',
 'BM25-BEAM10-ConFT-CBOW1',
 'BM25-BEAM10-ConFT-CBOW2',
 'BM25-BEAM10-ConFT-CBOW3',
 'BM25-BEAM10-ConFT-CBOW4',
 ],
    verbose = True,
    baseline=0
)

pt.Experiment:   0%|          | 0/17 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,BM25,0.742088,0.186139,0.861386,0.876238,0.900990,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BM25-BEAM10-ConW2V-SG1,0.707513,0.184158,0.836634,0.896040,0.910891,8.0,16.0,0.084201,1.0,...,0.566290,3.0,6.0,0.277400,5.0,3.0,0.287281,3.0,1.0,0.319724
2,BM25-BEAM10-ConW2V-SG2,0.715907,0.188119,0.846535,0.886139,0.905941,9.0,13.0,0.176797,1.0,...,0.319724,2.0,4.0,0.441315,4.0,3.0,0.619482,2.0,1.0,0.566290
3,BM25-BEAM10-ConW2V-SG3,0.707532,0.184158,0.836634,0.896040,0.910891,8.0,16.0,0.084373,1.0,...,0.566290,3.0,6.0,0.277400,5.0,3.0,0.287281,3.0,1.0,0.319724
4,BM25-BEAM10-ConW2V-SG4,0.715907,0.188119,0.846535,0.886139,0.905941,9.0,13.0,0.176797,1.0,...,0.319724,2.0,4.0,0.441315,4.0,3.0,0.619482,2.0,1.0,0.566290
5,BM25-BEAM10-ConW2V-CBOW1,0.682598,0.178218,0.821782,0.856436,0.891089,7.0,24.0,0.009094,2.0,...,0.158330,2.0,6.0,0.073477,3.0,5.0,0.287281,3.0,3.0,0.566290
6,BM25-BEAM10-ConW2V-CBOW2,0.691915,0.180198,0.821782,0.871287,0.891089,9.0,20.0,0.026947,1.0,...,0.181012,1.0,6.0,0.044941,2.0,2.0,0.707450,3.0,3.0,0.566290
7,BM25-BEAM10-ConW2V-CBOW3,0.693527,0.176238,0.826733,0.861386,0.905941,7.0,20.0,0.019775,0.0,...,0.024597,2.0,6.0,0.089635,2.0,4.0,0.319724,2.0,1.0,0.566290
8,BM25-BEAM10-ConW2V-CBOW4,0.701891,0.180198,0.821782,0.881188,0.900990,10.0,19.0,0.049621,0.0,...,0.083249,1.0,6.0,0.044941,2.0,1.0,0.566290,1.0,1.0,1.000000
9,BM25-BEAM10-ConFT-SG1,0.722868,0.176238,0.851485,0.876238,0.905941,8.0,18.0,0.280242,1.0,...,0.058402,2.0,3.0,0.482235,3.0,3.0,1.000000,3.0,1.0,0.707450


#### TF-IDF Beam 10

In [72]:
# Non-Contextualized
tfidfv11 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv12 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv21 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv22 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv31 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv32 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv41 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv42 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30

# Contextualized
tfidfv13 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv14 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv15 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv16 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30

tfidfv23 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv24 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv25 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv26 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30

tfidfv33 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv34 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv35 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv36 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30

tfidfv43 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv44 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv45 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
tfidfv46 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30

In [73]:
pt.Experiment(
    [
tfidfv10,
tfidfv11,
tfidfv12,
tfidfv21,
tfidfv22,
tfidfv31,
tfidfv32,
tfidfv41,
tfidfv42,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = ['TF-IDF',
 'TF-IDF-BEAM10-NoConW2V-SG1',
 'TF-IDF-BEAM10-NoConW2V-SG2',
 'TF-IDF-BEAM10-NoConW2V-CBOW1',
 'TF-IDF-BEAM10-NoConW2V-CBOW2',
 'TF-IDF-BEAM10-NoConFT-SG1',
 'TF-IDF-BEAM10-NoConFT-SG2',
 'TF-IDF-BEAM10-NoConFT-CBOW1',
 'TF-IDF-BEAM10-NoConFT-CBOW2'
 ],
    verbose=True,
    baseline=0
)

pt.Experiment:   0%|          | 0/9 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,TF-IDF,0.741401,0.190099,0.856436,0.876238,0.891089,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TF-IDF-BEAM10-NoConW2V-SG1,0.684077,0.178218,0.826733,0.886139,0.915842,8.0,20.0,0.015224,0.0,...,0.013564,3.0,7.0,0.202355,5.0,4.0,0.639676,4.0,0.0,0.058402
2,TF-IDF-BEAM10-NoConW2V-SG2,0.684077,0.180198,0.826733,0.886139,0.915842,8.0,20.0,0.015224,0.0,...,0.024597,3.0,7.0,0.202355,5.0,4.0,0.639676,4.0,0.0,0.058402
3,TF-IDF-BEAM10-NoConW2V-CBOW1,0.672527,0.176238,0.816832,0.856436,0.881188,5.0,26.0,0.003614,0.0,...,0.007509,2.0,6.0,0.073477,2.0,4.0,0.250179,2.0,3.0,0.482235
4,TF-IDF-BEAM10-NoConW2V-CBOW2,0.685202,0.172277,0.816832,0.866337,0.891089,7.0,22.0,0.011373,0.0,...,0.002308,2.0,6.0,0.073477,2.0,3.0,0.482235,2.0,2.0,1.000000
5,TF-IDF-BEAM10-NoConFT-SG1,0.711840,0.182178,0.841584,0.881188,0.900990,8.0,17.0,0.072212,0.0,...,0.044941,0.0,2.0,0.181012,2.0,1.0,0.566290,3.0,1.0,0.319724
6,TF-IDF-BEAM10-NoConFT-SG2,0.706890,0.182178,0.841584,0.881188,0.900990,8.0,18.0,0.044011,0.0,...,0.044941,0.0,2.0,0.181012,2.0,1.0,0.566290,3.0,1.0,0.319724
7,TF-IDF-BEAM10-NoConFT-CBOW1,0.719952,0.192079,0.861386,0.881188,0.896040,14.0,12.0,0.173876,3.0,...,0.656968,2.0,1.0,0.566290,2.0,2.0,0.707450,2.0,1.0,0.566290
8,TF-IDF-BEAM10-NoConFT-CBOW2,0.719952,0.192079,0.861386,0.881188,0.896040,14.0,12.0,0.173876,3.0,...,0.656968,2.0,1.0,0.566290,2.0,2.0,0.707450,2.0,1.0,0.566290


In [74]:
pt.Experiment(
    [
tfidfv10,
tfidfv13,
tfidfv14,
tfidfv15,
tfidfv16,
tfidfv23,
tfidfv24,
tfidfv25,
tfidfv26,
tfidfv33,
tfidfv34,
tfidfv35,
tfidfv36,
tfidfv43,
tfidfv44,
tfidfv45,
tfidfv46,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = ['TF-IDF',
'TF-IDF-BEAM10-ConW2V-SG1',
 'TF-IDF-BEAM10-ConW2V-SG2',
 'TF-IDF-BEAM10-ConW2V-SG3',
 'TF-IDF-BEAM10-ConW2V-SG4',
 'TF-IDF-BEAM10-ConW2V-CBOW1',
 'TF-IDF-BEAM10-ConW2V-CBOW2',
 'TF-IDF-BEAM10-ConW2V-CBOW3',
 'TF-IDF-BEAM10-ConW2V-CBOW4',
 'TF-IDF-BEAM10-ConFT-SG1',
 'TF-IDF-BEAM10-ConFT-SG2',
 'TF-IDF-BEAM10-ConFT-SG3',
 'TF-IDF-BEAM10-ConFT-SG4',
 'TF-IDF-BEAM10-ConFT-CBOW1',
 'TF-IDF-BEAM10-ConFT-CBOW2',
 'TF-IDF-BEAM10-ConFT-CBOW3',
 'TF-IDF-BEAM10-ConFT-CBOW4'
 ],
    verbose=True,
    baseline=0
)

pt.Experiment:   0%|          | 0/17 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,TF-IDF,0.741401,0.190099,0.856436,0.876238,0.891089,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TF-IDF-BEAM10-ConW2V-SG1,0.702253,0.180198,0.831683,0.886139,0.910891,8.0,16.0,0.065628,0.0,...,0.024597,3.0,6.0,0.277400,5.0,4.0,0.639676,3.0,0.0,0.102744
2,TF-IDF-BEAM10-ConW2V-SG2,0.708909,0.188119,0.836634,0.891089,0.910891,9.0,14.0,0.116590,0.0,...,0.319724,3.0,5.0,0.373717,4.0,2.0,0.441315,3.0,0.0,0.102744
3,TF-IDF-BEAM10-ConW2V-SG3,0.702253,0.182178,0.831683,0.886139,0.910891,8.0,16.0,0.065628,0.0,...,0.044941,3.0,6.0,0.277400,5.0,4.0,0.639676,3.0,0.0,0.102744
4,TF-IDF-BEAM10-ConW2V-SG4,0.708909,0.188119,0.836634,0.891089,0.910891,9.0,14.0,0.116590,0.0,...,0.319724,3.0,5.0,0.373717,4.0,2.0,0.441315,3.0,0.0,0.102744
5,TF-IDF-BEAM10-ConW2V-CBOW1,0.683780,0.178218,0.816832,0.856436,0.881188,7.0,24.0,0.009412,0.0,...,0.013564,2.0,6.0,0.073477,2.0,4.0,0.250179,2.0,3.0,0.482235
6,TF-IDF-BEAM10-ConW2V-CBOW2,0.689432,0.184158,0.821782,0.866337,0.886139,7.0,21.0,0.018030,0.0,...,0.083249,3.0,6.0,0.127253,1.0,2.0,0.416930,2.0,2.0,0.707450
7,TF-IDF-BEAM10-ConW2V-CBOW3,0.696930,0.176238,0.826733,0.866337,0.896040,7.0,20.0,0.029544,0.0,...,0.007509,2.0,5.0,0.134334,2.0,3.0,0.482235,3.0,2.0,0.764686
8,TF-IDF-BEAM10-ConW2V-CBOW4,0.700221,0.184158,0.836634,0.876238,0.891089,8.0,19.0,0.041671,0.0,...,0.083249,4.0,5.0,0.348317,1.0,1.0,1.000000,2.0,2.0,1.000000
9,TF-IDF-BEAM10-ConFT-SG1,0.724637,0.182178,0.846535,0.886139,0.896040,8.0,16.0,0.310016,0.0,...,0.044941,1.0,2.0,0.416930,3.0,1.0,0.319724,2.0,1.0,0.566290


#### TF-IDF Beam 7

In [75]:
# Non-Contextualized
tfidfv11 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv12 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv21 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv22 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv31 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv32 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv41 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv42 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30

# Contextualized
tfidfv13 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv14 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv15 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv16 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30

tfidfv23 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv24 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv25 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv26 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30

tfidfv33 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv34 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv35 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv36 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30

tfidfv43 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv44 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv45 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
tfidfv46 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30

In [76]:
pt.Experiment(
    [
tfidfv10,
tfidfv11,
tfidfv12,
tfidfv21,
tfidfv22,
tfidfv31,
tfidfv32,
tfidfv41,
tfidfv42,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = ['TF-IDF',
 'TF-IDF-BEAM7-NoConW2V-SG1',
 'TF-IDF-BEAM7-NoConW2V-SG2',
 'TF-IDF-BEAM7-NoConW2V-CBOW1',
 'TF-IDF-BEAM7-NoConW2V-CBOW2',
 'TF-IDF-BEAM7-NoConFT-SG1',
 'TF-IDF-BEAM7-NoConFT-SG2',
 'TF-IDF-BEAM7-NoConFT-CBOW1',
 'TF-IDF-BEAM7-NoConFT-CBOW2'],
    verbose=True,
    baseline=0
)

pt.Experiment:   0%|          | 0/9 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,TF-IDF,0.741401,0.190099,0.856436,0.876238,0.891089,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TF-IDF-BEAM7-NoConW2V-SG1,0.686098,0.180198,0.836634,0.876238,0.905941,9.0,21.0,0.015358,0.0,...,0.024597,4.0,7.0,0.373717,5.0,5.0,1.000000,4.0,2.0,0.319724
2,TF-IDF-BEAM7-NoConW2V-SG2,0.686098,0.180198,0.836634,0.876238,0.905941,9.0,21.0,0.015358,0.0,...,0.024597,4.0,7.0,0.373717,5.0,5.0,1.000000,4.0,2.0,0.319724
3,TF-IDF-BEAM7-NoConW2V-CBOW1,0.668361,0.172277,0.831683,0.846535,0.881188,5.0,28.0,0.001856,0.0,...,0.002308,3.0,5.0,0.227022,2.0,6.0,0.109176,3.0,4.0,0.529762
4,TF-IDF-BEAM7-NoConW2V-CBOW2,0.670182,0.176238,0.816832,0.856436,0.891089,5.0,23.0,0.001714,0.0,...,0.007509,3.0,7.0,0.088139,2.0,5.0,0.207485,2.0,2.0,1.000000
5,TF-IDF-BEAM7-NoConFT-SG1,0.712899,0.182178,0.846535,0.866337,0.881188,8.0,17.0,0.067870,0.0,...,0.044941,1.0,2.0,0.416930,2.0,4.0,0.416930,3.0,4.0,0.529762
6,TF-IDF-BEAM7-NoConFT-SG2,0.712899,0.182178,0.846535,0.866337,0.881188,8.0,17.0,0.067870,0.0,...,0.044941,1.0,2.0,0.416930,2.0,4.0,0.416930,3.0,4.0,0.529762
7,TF-IDF-BEAM7-NoConFT-CBOW1,0.722946,0.192079,0.851485,0.861386,0.891089,10.0,12.0,0.181243,3.0,...,0.656968,2.0,3.0,0.656968,1.0,4.0,0.181012,2.0,2.0,1.000000
8,TF-IDF-BEAM7-NoConFT-CBOW2,0.722946,0.192079,0.851485,0.861386,0.891089,10.0,12.0,0.181243,3.0,...,0.656968,2.0,3.0,0.656968,1.0,4.0,0.181012,2.0,2.0,1.000000


In [77]:
pt.Experiment(
    [
tfidfv10,
tfidfv13,
tfidfv14,
tfidfv15,
tfidfv16,
tfidfv23,
tfidfv24,
tfidfv25,
tfidfv26,
tfidfv33,
tfidfv34,
tfidfv35,
tfidfv36,
tfidfv43,
tfidfv44,
tfidfv45,
tfidfv46,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = ['TF-IDF',
'TF-IDF-BEAM7-ConW2V-SG1',
 'TF-IDF-BEAM7-ConW2V-SG2',
 'TF-IDF-BEAM7-ConW2V-SG3',
 'TF-IDF-BEAM7-ConW2V-SG4',
 'TF-IDF-BEAM7-ConW2V-CBOW1',
 'TF-IDF-BEAM7-ConW2V-CBOW2',
 'TF-IDF-BEAM7-ConW2V-CBOW3',
 'TF-IDF-BEAM7-ConW2V-CBOW4',
 'TF-IDF-BEAM7-ConFT-SG1',
 'TF-IDF-BEAM7-ConFT-SG2',
 'TF-IDF-BEAM7-ConFT-SG3',
 'TF-IDF-BEAM-ConFT-SG4',
 'TF-IDF-BEAM7-ConFT-CBOW1',
 'TF-IDF-BEAM7-ConFT-CBOW2',
 'TF-IDF-BEAM7-ConFT-CBOW3',
 'TF-IDF-BEAM7-ConFT-CBOW4'
 ],
    verbose=True,
    baseline=0
)

pt.Experiment:   0%|          | 0/17 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,TF-IDF,0.741401,0.190099,0.856436,0.876238,0.891089,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TF-IDF-BEAM7-ConW2V-SG1,0.708053,0.182178,0.841584,0.871287,0.900990,9.0,18.0,0.122252,0.0,...,0.044941,4.0,6.0,0.494022,4.0,5.0,0.797705,3.0,2.0,0.482235
2,TF-IDF-BEAM7-ConW2V-SG2,0.714064,0.186139,0.846535,0.876238,0.900990,9.0,16.0,0.193245,0.0,...,0.158330,3.0,4.0,0.619482,3.0,3.0,1.000000,3.0,2.0,0.482235
3,TF-IDF-BEAM7-ConW2V-SG3,0.708053,0.182178,0.841584,0.871287,0.900990,9.0,18.0,0.122252,0.0,...,0.044941,4.0,6.0,0.494022,4.0,5.0,0.797705,3.0,2.0,0.482235
4,TF-IDF-BEAM7-ConW2V-SG4,0.714064,0.186139,0.846535,0.876238,0.900990,9.0,16.0,0.193245,0.0,...,0.158330,3.0,4.0,0.619482,3.0,3.0,1.000000,3.0,2.0,0.482235
5,TF-IDF-BEAM7-ConW2V-CBOW1,0.679665,0.174257,0.831683,0.846535,0.881188,6.0,25.0,0.005245,0.0,...,0.004163,3.0,5.0,0.227022,2.0,6.0,0.109176,3.0,4.0,0.529762
6,TF-IDF-BEAM7-ConW2V-CBOW2,0.693784,0.180198,0.821782,0.856436,0.891089,5.0,20.0,0.023189,0.0,...,0.024597,2.0,6.0,0.089635,1.0,4.0,0.158330,3.0,2.0,1.000000
7,TF-IDF-BEAM7-ConW2V-CBOW3,0.686847,0.178218,0.826733,0.856436,0.891089,6.0,21.0,0.012036,0.0,...,0.013564,3.0,6.0,0.158330,2.0,5.0,0.207485,2.0,2.0,1.000000
8,TF-IDF-BEAM7-ConW2V-CBOW4,0.699991,0.182178,0.831683,0.866337,0.891089,7.0,18.0,0.044684,0.0,...,0.044941,2.0,5.0,0.166650,1.0,3.0,0.319724,2.0,2.0,1.000000
9,TF-IDF-BEAM7-ConFT-SG1,0.729069,0.182178,0.846535,0.871287,0.876238,8.0,15.0,0.434590,0.0,...,0.044941,1.0,2.0,0.416930,3.0,4.0,0.707450,2.0,4.0,0.319724


#### BM25 Beam 7

In [78]:
# Non-Contextualized
bm25v11 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v12 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v21 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v22 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v31 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v32 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v41 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v42 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30

# Contextualized
bm25v13 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v14 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v15 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v16 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30

bm25v23 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v24 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v25 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v26 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30

bm25v33 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v34 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v35 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v36 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30

bm25v43 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v44 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v45 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30
bm25v46 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30

In [79]:
pt.Experiment(
    [
bm25v10,
bm25v11,
bm25v12,
bm25v21,
bm25v22,
bm25v31,
bm25v32,
bm25v41,
bm25v42,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = ['BM25',
 'BM25-BEAM7-NoConW2V-SG1',
 'BM25-BEAM7-NoConW2V-SG2',
 'BM25-BEAM7-NoConW2V-CBOW1',
 'BM25-BEAM7-NoConW2V-CBOW2',
 'BM25-BEAM7-NoConFT-SG1',
 'BM25-BEAM7-NoConFT-SG2',
 'BM25-BEAM7-NoConFT-CBOW1',
 'BM25-BEAM7-NoConFT-CBOW2'],
    verbose = True,
    baseline=0
)

pt.Experiment:   0%|          | 0/9 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,BM25,0.742088,0.186139,0.861386,0.876238,0.900990,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BM25-BEAM7-NoConW2V-SG1,0.682434,0.182178,0.841584,0.886139,0.900990,8.0,22.0,0.006257,2.0,...,0.416930,3.0,6.0,0.348317,4.0,3.0,0.529762,3.0,2.0,1.000000
2,BM25-BEAM7-NoConW2V-SG2,0.682434,0.182178,0.841584,0.886139,0.900990,8.0,22.0,0.006257,2.0,...,0.416930,3.0,6.0,0.348317,4.0,3.0,0.529762,3.0,2.0,1.000000
3,BM25-BEAM7-NoConW2V-CBOW1,0.673360,0.172277,0.831683,0.851485,0.881188,6.0,26.0,0.003349,1.0,...,0.018867,2.0,5.0,0.134334,2.0,5.0,0.166650,3.0,5.0,0.287281
4,BM25-BEAM7-NoConW2V-CBOW2,0.677806,0.170297,0.816832,0.866337,0.896040,6.0,23.0,0.005197,0.0,...,0.004163,2.0,7.0,0.049027,2.0,4.0,0.566290,2.0,2.0,0.707450
5,BM25-BEAM7-NoConFT-SG1,0.714324,0.180198,0.846535,0.866337,0.905941,8.0,18.0,0.069685,1.0,...,0.181012,2.0,4.0,0.319724,1.0,3.0,0.319724,3.0,1.0,0.707450
6,BM25-BEAM7-NoConFT-SG2,0.709373,0.180198,0.846535,0.866337,0.905941,8.0,19.0,0.041529,1.0,...,0.181012,2.0,4.0,0.319724,1.0,3.0,0.319724,3.0,1.0,0.707450
7,BM25-BEAM7-NoConFT-CBOW1,0.726469,0.192079,0.851485,0.866337,0.896040,8.0,13.0,0.308131,5.0,...,0.258873,2.0,4.0,0.416930,1.0,3.0,0.319724,3.0,3.0,0.740688
8,BM25-BEAM7-NoConFT-CBOW2,0.726469,0.192079,0.851485,0.866337,0.896040,8.0,13.0,0.308131,5.0,...,0.258873,2.0,4.0,0.416930,1.0,3.0,0.319724,3.0,3.0,0.740688


In [80]:
pt.Experiment(
    [
bm25v10,
bm25v13,
bm25v14,
bm25v15,
bm25v16,
bm25v23,
bm25v24,
bm25v25,
bm25v26,
bm25v33,
bm25v34,
bm25v35,
bm25v36,
bm25v43,
bm25v44,
bm25v45,
bm25v46,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = ['BM25',
 'BM25-BEAM7-ConW2V-SG1',
 'BM25-BEAM7-ConW2V-SG2',
 'BM25-BEAM7-ConW2V-SG3',
 'BM25-BEAM7-ConW2V-SG4',
 'BM25-BEAM7-ConW2V-CBOW1',
 'BM25-BEAM7-ConW2V-CBOW2',
 'BM25-BEAM7-ConW2V-CBOW3',
 'BM25-BEAM7-ConW2V-CBOW4',
 'BM25-BEAM7-ConFT-SG1',
 'BM25-BEAM7-ConFT-SG2',
 'BM25-BEAM7-ConFT-SG3',
 'BM25-BEAM7-ConFT-SG4',
 'BM25-BEAM7-ConFT-CBOW1',
 'BM25-BEAM7-ConFT-CBOW2',
 'BM25-BEAM7-ConFT-CBOW3',
 'BM25-BEAM7-ConFT-CBOW4',
 ],
    verbose = True,
    baseline=0
)

pt.Experiment:   0%|          | 0/17 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,BM25,0.742088,0.186139,0.861386,0.876238,0.900990,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BM25-BEAM7-ConW2V-SG1,0.704405,0.184158,0.841584,0.881188,0.896040,8.0,19.0,0.063993,2.0,...,0.656968,3.0,6.0,0.348317,3.0,3.0,0.740688,2.0,2.0,0.707450
2,BM25-BEAM7-ConW2V-SG2,0.710554,0.190099,0.846535,0.881188,0.896040,8.0,17.0,0.110301,2.0,...,0.158330,2.0,4.0,0.441315,2.0,2.0,0.707450,2.0,2.0,0.707450
3,BM25-BEAM7-ConW2V-SG3,0.704405,0.184158,0.841584,0.881188,0.896040,8.0,19.0,0.063993,2.0,...,0.656968,3.0,6.0,0.348317,3.0,3.0,0.740688,2.0,2.0,0.707450
4,BM25-BEAM7-ConW2V-SG4,0.710554,0.190099,0.846535,0.881188,0.896040,8.0,17.0,0.110301,2.0,...,0.158330,2.0,4.0,0.441315,2.0,2.0,0.707450,2.0,2.0,0.707450
5,BM25-BEAM7-ConW2V-CBOW1,0.683358,0.174257,0.831683,0.851485,0.881188,7.0,24.0,0.007977,1.0,...,0.033206,2.0,5.0,0.134334,2.0,5.0,0.166650,3.0,5.0,0.287281
6,BM25-BEAM7-ConW2V-CBOW2,0.693902,0.180198,0.821782,0.866337,0.891089,6.0,20.0,0.022297,0.0,...,0.083249,1.0,6.0,0.044941,1.0,2.0,0.416930,3.0,3.0,0.566290
7,BM25-BEAM7-ConW2V-CBOW3,0.691667,0.174257,0.826733,0.866337,0.896040,6.0,20.0,0.017994,0.0,...,0.013564,2.0,6.0,0.089635,2.0,4.0,0.566290,2.0,2.0,0.707450
8,BM25-BEAM7-ConW2V-CBOW4,0.701819,0.182178,0.836634,0.886139,0.891089,7.0,18.0,0.045195,0.0,...,0.158330,1.0,4.0,0.132360,2.0,1.0,0.416930,1.0,2.0,0.416930
9,BM25-BEAM7-ConFT-SG1,0.727187,0.180198,0.846535,0.871287,0.905941,8.0,16.0,0.340777,1.0,...,0.181012,2.0,4.0,0.319724,2.0,3.0,0.656968,3.0,1.0,0.707450


#### BM25 Random 5

In [81]:
# Non-Contextualized
bm25v11 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v12 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v21 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v22 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v31 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v32 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v41 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v42 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30

# Contextualized
bm25v13 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v14 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v15 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v16 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30

bm25v23 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v24 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v25 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v26 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30

bm25v33 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v34 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v35 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v36 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30

bm25v43 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v44 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v45 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30
bm25v46 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30

In [82]:
pt.Experiment(
    [
bm25v10,
bm25v11,
bm25v12,
bm25v21,
bm25v22,
bm25v31,
bm25v32,
bm25v41,
bm25v42
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
#     names = ['BM25',
#  'BM25-Rand5-NoConW2V-SG1',
#  'BM25-Rand5-NoConW2V-SG2',
#  'BM25-Rand5-NoConW2V-CBOW1',
#  'BM25-Rand5-NoConW2V-CBOW2',
#  'BM25-Rand5-NoConFT-SG1',
#  'BM25-Rand5-NoConFT-SG2',
#  'BM25-Rand5-NoConFT-CBOW1',
#  'BM25-Rand5-NoConFT-CBOW2'],
    verbose=True,
    baseline=0
)

pt.Experiment:   0%|          | 0/9 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,"RankCutoff(BR(BM25), 30)",0.742088,0.186139,0.861386,0.876238,0.900990,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.689310,0.176238,0.826733,0.871287,0.900990,9.0,24.0,0.029052,1.0,...,0.058402,1.0,6.0,0.051732,4.0,6.0,0.819851,3.0,3.0,1.000000
2,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.689360,0.176238,0.826733,0.871287,0.905941,9.0,24.0,0.029201,1.0,...,0.058402,1.0,6.0,0.051732,4.0,6.0,0.819851,3.0,2.0,0.656968
3,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.676636,0.174257,0.821782,0.856436,0.891089,9.0,26.0,0.007293,2.0,...,0.057385,1.0,6.0,0.044941,3.0,5.0,0.287281,3.0,3.0,0.566290
4,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.687860,0.174257,0.826733,0.856436,0.881188,10.0,26.0,0.026084,1.0,...,0.033206,1.0,5.0,0.070496,1.0,4.0,0.158330,1.0,3.0,0.207485
5,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.724667,0.180198,0.836634,0.871287,0.905941,8.0,20.0,0.299482,2.0,...,0.258873,0.0,4.0,0.058402,1.0,2.0,0.566290,4.0,2.0,0.740688
6,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.719717,0.180198,0.836634,0.871287,0.905941,8.0,21.0,0.200792,2.0,...,0.258873,0.0,4.0,0.058402,1.0,2.0,0.566290,4.0,2.0,0.740688
7,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.732868,0.184158,0.846535,0.876238,0.900990,9.0,15.0,0.576347,2.0,...,0.656968,2.0,5.0,0.258873,1.0,1.0,1.000000,4.0,3.0,1.000000
8,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.732868,0.184158,0.846535,0.876238,0.900990,9.0,15.0,0.576347,2.0,...,0.656968,2.0,5.0,0.258873,1.0,1.0,1.000000,4.0,3.0,1.000000


In [83]:
pt.Experiment(
    [
bm25v10,
bm25v13,
bm25v14,
bm25v15,
bm25v16,
bm25v23,
bm25v24,
bm25v25,
bm25v26,
bm25v33,
bm25v34,
bm25v35,
bm25v36,
bm25v43,
bm25v44,
bm25v45,
bm25v46,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
#     names = ['BM25',
#  'BM25-Rand5-ConW2V-SG1',
#  'BM25-Rand5-ConW2V-SG2',
#  'BM25-Rand5-ConW2V-CBOW1',
#  'BM25-Rand5-ConW2V-CBOW2',
#  'BM25-Rand5-ConFT-SG1',
#  'BM25-Rand5-ConFT-SG2',
#  'BM25-Rand5-ConFT-CBOW1',
#  'BM25-Rand5-ConFT-CBOW2'],
    verbose=True,
    baseline=0
)

pt.Experiment:   0%|          | 0/17 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,"RankCutoff(BR(BM25), 30)",0.742088,0.186139,0.861386,0.876238,0.900990,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.704225,0.180198,0.826733,0.871287,0.900990,9.0,22.0,0.085876,1.0,...,0.181012,1.0,6.0,0.051732,4.0,6.0,0.819851,3.0,3.0,1.000000
2,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.716104,0.184158,0.836634,0.876238,0.900990,10.0,19.0,0.226498,1.0,...,0.566290,1.0,4.0,0.132360,3.0,4.0,1.000000,2.0,2.0,1.000000
3,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.704275,0.180198,0.826733,0.871287,0.905941,9.0,22.0,0.086281,1.0,...,0.181012,1.0,6.0,0.051732,4.0,6.0,0.819851,3.0,2.0,0.656968
4,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.716130,0.184158,0.836634,0.876238,0.900990,10.0,19.0,0.226957,1.0,...,0.566290,1.0,4.0,0.132360,3.0,4.0,1.000000,2.0,2.0,1.000000
5,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.686583,0.176238,0.821782,0.866337,0.891089,9.0,24.0,0.018080,2.0,...,0.095751,1.0,6.0,0.044941,4.0,5.0,0.639676,3.0,3.0,0.566290
6,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.698978,0.182178,0.821782,0.876238,0.891089,9.0,21.0,0.056358,1.0,...,0.319724,1.0,6.0,0.044941,2.0,2.0,1.000000,3.0,3.0,0.566290
7,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.697491,0.176238,0.836634,0.856436,0.881188,10.0,24.0,0.056672,1.0,...,0.058402,1.0,4.0,0.132360,1.0,4.0,0.158330,1.0,3.0,0.207485
8,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.709735,0.184158,0.836634,0.876238,0.881188,10.0,21.0,0.148706,1.0,...,0.566290,1.0,4.0,0.132360,0.0,0.0,NaN,1.0,3.0,0.207485
9,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.741336,0.180198,0.836634,0.876238,0.905941,9.0,17.0,0.966550,2.0,...,0.258873,0.0,4.0,0.058402,2.0,2.0,1.000000,4.0,2.0,0.740688


#### TFIDF Random 5

In [84]:
# Non-Contextualized
tfidfv11 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv12 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv21 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv22 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv31 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv32 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv41 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv42 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30

# Contextualized
tfidfv13 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv14 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv15 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv16 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30

tfidfv23 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv24 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv25 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv26 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30

tfidfv33 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv34 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv35 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv36 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30

tfidfv43 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv44 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv45 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
tfidfv46 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30

In [85]:
pt.Experiment(
    [
tfidfv10,
tfidfv11,
tfidfv12,
tfidfv21,
tfidfv22,
tfidfv31,
tfidfv32,
tfidfv41,
tfidfv42,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
#     names = ['TF-IDF',
#  'TF-IDF-BEAM7-NoConW2V-SG1',
#  'TF-IDF-BEAM7-NoConW2V-SG2',
#  'TF-IDF-BEAM7-NoConW2V-CBOW1',
#  'TF-IDF-BEAM7-NoConW2V-CBOW2',
#  'TF-IDF-BEAM7-NoConFT-SG1',
#  'TF-IDF-BEAM7-NoConFT-SG2',
#  'TF-IDF-BEAM7-NoConFT-CBOW1',
#  'TF-IDF-BEAM7-NoConFT-CBOW2'],
    verbose=True,
    baseline=0
)

pt.Experiment:   0%|          | 0/9 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,"RankCutoff(BR(TF_IDF), 30)",0.741401,0.190099,0.856436,0.876238,0.891089,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.689734,0.178218,0.821782,0.871287,0.905941,9.0,24.0,0.033812,0.0,...,0.033206,2.0,7.0,0.070496,5.0,7.0,0.828506,4.0,2.0,0.319724
2,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.689763,0.178218,0.821782,0.871287,0.905941,9.0,24.0,0.033911,0.0,...,0.033206,2.0,7.0,0.070496,5.0,7.0,0.828506,4.0,2.0,0.319724
3,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.674549,0.174257,0.826733,0.861386,0.886139,8.0,28.0,0.008591,0.0,...,0.004163,2.0,5.0,0.134334,4.0,6.0,0.494022,3.0,3.0,0.740688
4,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.682163,0.172277,0.811881,0.856436,0.881188,9.0,25.0,0.013071,0.0,...,0.006059,2.0,7.0,0.049027,2.0,5.0,0.207485,1.0,2.0,0.416930
5,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.716733,0.180198,0.826733,0.871287,0.905941,8.0,19.0,0.143061,2.0,...,0.095751,0.0,5.0,0.033206,2.0,3.0,0.656968,4.0,1.0,0.181012
6,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.711783,0.180198,0.826733,0.871287,0.905941,8.0,20.0,0.091075,2.0,...,0.095751,0.0,5.0,0.033206,2.0,3.0,0.656968,4.0,1.0,0.181012
7,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.731577,0.184158,0.846535,0.881188,0.896040,12.0,15.0,0.560646,2.0,...,0.258873,1.0,3.0,0.319724,3.0,3.0,0.740688,3.0,2.0,0.656968
8,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.731577,0.184158,0.846535,0.881188,0.896040,12.0,15.0,0.560646,2.0,...,0.258873,1.0,3.0,0.319724,3.0,3.0,0.740688,3.0,2.0,0.656968


In [86]:
pt.Experiment(
    [
tfidfv10,
tfidfv13,
tfidfv14,
tfidfv15,
tfidfv16,
tfidfv23,
tfidfv24,
tfidfv25,
tfidfv26,
tfidfv33,
tfidfv34,
tfidfv35,
tfidfv36,
tfidfv43,
tfidfv44,
tfidfv45,
tfidfv46,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
#     names = ['TF-IDF',
#  'TF-IDF-Rand5-NoConW2V-SG1',
#  'TF-IDF-Rand5-NoConW2V-SG2',
#  'TF-IDF-Rand5-NoConW2V-CBOW1',
#  'TF-IDF-Rand5-NoConW2V-CBOW2',
#  'TF-IDF-Rand5-NoConFT-SG1',
#  'TF-IDF-Rand5-NoConFT-SG2',
#  'TF-IDF-Rand5-NoConFT-CBOW1',
#  'TF-IDF-Rand5-NoConFT-CBOW2'],
    verbose=True,
    baseline=0
)

pt.Experiment:   0%|          | 0/17 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,"RankCutoff(BR(TF_IDF), 30)",0.741401,0.190099,0.856436,0.876238,0.891089,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.704751,0.180198,0.826733,0.871287,0.900990,9.0,22.0,0.098305,0.0,...,0.058402,2.0,6.0,0.109176,5.0,7.0,0.828506,3.0,2.0,0.482235
2,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.716595,0.184158,0.836634,0.876238,0.900990,10.0,18.0,0.254511,0.0,...,0.181012,2.0,4.0,0.250179,4.0,5.0,1.000000,3.0,2.0,0.482235
3,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.704779,0.180198,0.826733,0.871287,0.900990,9.0,22.0,0.098566,0.0,...,0.058402,2.0,6.0,0.109176,5.0,7.0,0.828506,3.0,2.0,0.482235
4,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.716627,0.184158,0.836634,0.876238,0.900990,10.0,18.0,0.255121,0.0,...,0.181012,2.0,4.0,0.250179,4.0,5.0,1.000000,3.0,2.0,0.482235
5,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.685176,0.178218,0.826733,0.861386,0.886139,8.0,25.0,0.020771,0.0,...,0.013564,2.0,5.0,0.134334,4.0,6.0,0.494022,3.0,3.0,0.740688
6,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.698396,0.180198,0.826733,0.876238,0.891089,9.0,22.0,0.068285,0.0,...,0.024597,3.0,6.0,0.158330,3.0,3.0,1.000000,3.0,2.0,1.000000
7,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.692728,0.174257,0.821782,0.856436,0.881188,9.0,23.0,0.030440,0.0,...,0.010704,2.0,6.0,0.089635,2.0,5.0,0.207485,1.0,2.0,0.416930
8,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.704444,0.180198,0.836634,0.871287,0.881188,9.0,20.0,0.088896,0.0,...,0.058402,3.0,5.0,0.287281,1.0,2.0,0.566290,1.0,2.0,0.416930
9,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.735976,0.182178,0.826733,0.876238,0.900990,9.0,16.0,0.758269,2.0,...,0.158330,0.0,5.0,0.033206,3.0,3.0,1.000000,3.0,1.0,0.319724


#### TFIDF Random 7

In [87]:
# Non-Contextualized
tfidfv11 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv12 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv21 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv22 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv31 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv32 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv41 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv42 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30

# Contextualized
tfidfv13 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv14 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv15 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv16 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30

tfidfv23 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv24 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv25 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv26 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30

tfidfv33 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv34 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv35 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv36 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30

tfidfv43 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv44 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv45 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
tfidfv46 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30

In [88]:
pt.Experiment(
    [
tfidfv10,
tfidfv11,
tfidfv12,
tfidfv21,
tfidfv22,
tfidfv31,
tfidfv32,
tfidfv41,
tfidfv42,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = ['TF-IDF',
 'TF-IDF-Rand5-NoConW2V-SG1',
 'TF-IDF-Rand5-NoConW2V-SG2',
 'TF-IDF-Rand5-NoConW2V-CBOW1',
 'TF-IDF-Rand5-NoConW2V-CBOW2',
 'TF-IDF-Rand5-NoConFT-SG1',
 'TF-IDF-Rand5-NoConFT-SG2',
 'TF-IDF-Rand5-NoConFT-CBOW1',
 'TF-IDF-Rand5-NoConFT-CBOW2'],
    verbose=True,
    baseline=0
)

pt.Experiment:   0%|          | 0/9 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,TF-IDF,0.741401,0.190099,0.856436,0.876238,0.891089,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TF-IDF-Rand5-NoConW2V-SG1,0.684005,0.178218,0.826733,0.861386,0.900990,9.0,24.0,0.017092,0.0,...,0.033206,3.0,7.0,0.134334,4.0,6.0,0.408081,4.0,2.0,0.566290
2,TF-IDF-Rand5-NoConW2V-SG2,0.684005,0.178218,0.826733,0.861386,0.900990,9.0,24.0,0.017092,0.0,...,0.033206,3.0,7.0,0.134334,4.0,6.0,0.408081,4.0,2.0,0.566290
3,TF-IDF-Rand5-NoConW2V-CBOW1,0.666168,0.172277,0.826733,0.856436,0.876238,4.0,29.0,0.001676,0.0,...,0.002308,2.0,5.0,0.134334,3.0,5.0,0.287281,3.0,4.0,0.408081
4,TF-IDF-Rand5-NoConW2V-CBOW2,0.668938,0.170297,0.826733,0.861386,0.876238,7.0,27.0,0.002327,0.0,...,0.003420,2.0,5.0,0.134334,2.0,4.0,0.319724,2.0,3.0,0.368320
5,TF-IDF-Rand5-NoConFT-SG1,0.706892,0.182178,0.846535,0.876238,0.886139,6.0,22.0,0.037925,2.0,...,0.158330,1.0,2.0,0.416930,3.0,3.0,1.000000,3.0,3.0,0.740688
6,TF-IDF-Rand5-NoConFT-SG2,0.701941,0.182178,0.846535,0.876238,0.886139,6.0,23.0,0.022583,2.0,...,0.158330,1.0,2.0,0.416930,3.0,3.0,1.000000,3.0,3.0,0.740688
7,TF-IDF-Rand5-NoConFT-CBOW1,0.728481,0.188119,0.856436,0.866337,0.891089,10.0,16.0,0.373559,3.0,...,0.707450,2.0,2.0,1.000000,2.0,4.0,0.416930,2.0,2.0,1.000000
8,TF-IDF-Rand5-NoConFT-CBOW2,0.728481,0.188119,0.856436,0.866337,0.891089,10.0,16.0,0.373559,3.0,...,0.707450,2.0,2.0,1.000000,2.0,4.0,0.416930,2.0,2.0,1.000000


In [89]:
pt.Experiment(
    [
tfidfv10,
tfidfv13,
tfidfv14,
tfidfv15,
tfidfv16,
tfidfv23,
tfidfv24,
tfidfv25,
tfidfv26,
tfidfv33,
tfidfv34,
tfidfv35,
tfidfv36,
tfidfv43,
tfidfv44,
tfidfv45,
tfidfv46,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
#     names = ['TF-IDF',
#  'TF-IDF-Rand5-ConW2V-SG1',
#  'TF-IDF-Rand5-ConW2V-SG2',
#  'TF-IDF-Rand5-ConW2V-CBOW1',
#  'TF-IDF-Rand5-ConW2V-CBOW2',
#  'TF-IDF-Rand5-ConFT-SG1',
#  'TF-IDF-Rand5-ConFT-SG2',
#  'TF-IDF-Rand5-ConFT-CBOW1',
#  'TF-IDF-Rand5-ConFT-CBOW2'],
    verbose=True,
    baseline=0
)

pt.Experiment:   0%|          | 0/17 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,"RankCutoff(BR(TF_IDF), 30)",0.741401,0.190099,0.856436,0.876238,0.891089,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.702117,0.180198,0.831683,0.856436,0.900990,9.0,21.0,0.093380,0.0,...,0.058402,3.0,6.0,0.198192,3.0,6.0,0.250179,4.0,2.0,0.566290
2,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.708072,0.184158,0.836634,0.861386,0.896040,12.0,17.0,0.138506,0.0,...,0.181012,3.0,5.0,0.287281,2.0,4.0,0.319724,3.0,2.0,0.764686
3,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.702117,0.180198,0.831683,0.856436,0.900990,9.0,21.0,0.093380,0.0,...,0.058402,3.0,6.0,0.198192,3.0,6.0,0.250179,4.0,2.0,0.566290
4,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.708072,0.184158,0.836634,0.861386,0.896040,12.0,17.0,0.138506,0.0,...,0.181012,3.0,5.0,0.287281,2.0,4.0,0.319724,3.0,2.0,0.764686
5,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.677498,0.176238,0.826733,0.856436,0.876238,4.0,26.0,0.004936,0.0,...,0.007509,2.0,5.0,0.134334,3.0,5.0,0.287281,3.0,4.0,0.408081
6,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.689469,0.178218,0.831683,0.866337,0.881188,6.0,24.0,0.019833,0.0,...,0.013564,3.0,5.0,0.227022,2.0,3.0,0.482235,3.0,3.0,0.566290
7,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.679232,0.174257,0.836634,0.861386,0.876238,7.0,25.0,0.005974,0.0,...,0.010704,2.0,4.0,0.250179,2.0,4.0,0.319724,2.0,3.0,0.368320
8,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.689277,0.180198,0.841584,0.861386,0.891089,8.0,23.0,0.018954,0.0,...,0.058402,3.0,4.0,0.408081,1.0,3.0,0.258873,2.0,1.0,1.000000
9,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.721669,0.182178,0.851485,0.876238,0.881188,7.0,20.0,0.277544,2.0,...,0.158330,2.0,2.0,0.707450,3.0,3.0,1.000000,2.0,3.0,0.482235


#### BM25 Random 7

In [90]:
# Non-Contextualized
bm25v11 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v12 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v21 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v22 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v31 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v32 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v41 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v42 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30

# Contextualized
bm25v13 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v14 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v15 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v16 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30

bm25v23 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v24 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v25 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v26 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30

bm25v33 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v34 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v35 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v36 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30

bm25v43 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v44 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v45 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30
bm25v46 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30

In [91]:
pt.Experiment(
    [
bm25v10,
bm25v11,
bm25v12,
bm25v21,
bm25v22,
bm25v31,
bm25v32,
bm25v41,
bm25v42,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = ['BM25',
 'BM25-Rand10-NoConW2V-SG1',
 'BM25-Rand10-NoConW2V-SG2',
 'BM25-Rand10-NoConW2V-CBOW1',
 'BM25-Rand10-NoConW2V-CBOW2',
 'BM25-Rand10-NoConFT-SG1',
 'BM25-Rand10-NoConFT-SG2',
 'BM25-Rand10-NoConFT-CBOW1',
 'BM25-Rand10-NoConFT-CBOW2'],
    verbose=True,
    baseline=0
)

pt.Experiment:   0%|          | 0/9 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,BM25,0.742088,0.186139,0.861386,0.876238,0.900990,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BM25-Rand10-NoConW2V-SG1,0.685180,0.174257,0.831683,0.881188,0.905941,9.0,25.0,0.017932,0.0,...,0.013564,2.0,6.0,0.109176,4.0,4.0,0.764686,3.0,2.0,0.656968
2,BM25-Rand10-NoConW2V-SG2,0.685212,0.174257,0.831683,0.881188,0.905941,9.0,25.0,0.017997,0.0,...,0.013564,2.0,6.0,0.109176,4.0,4.0,0.764686,3.0,2.0,0.656968
3,BM25-Rand10-NoConW2V-CBOW1,0.663045,0.168317,0.831683,0.851485,0.881188,4.0,30.0,0.001044,0.0,...,0.002308,3.0,6.0,0.158330,2.0,5.0,0.166650,3.0,4.0,0.319724
4,BM25-Rand10-NoConW2V-CBOW2,0.664977,0.170297,0.831683,0.861386,0.886139,4.0,29.0,0.000609,0.0,...,0.004163,2.0,5.0,0.134334,1.0,3.0,0.258873,2.0,3.0,0.368320
5,BM25-Rand10-NoConFT-SG1,0.705703,0.182178,0.836634,0.876238,0.891089,6.0,23.0,0.024497,2.0,...,0.416930,0.0,4.0,0.058402,2.0,2.0,1.000000,3.0,3.0,0.566290
6,BM25-Rand10-NoConFT-SG2,0.705703,0.182178,0.836634,0.876238,0.891089,6.0,23.0,0.024497,2.0,...,0.416930,0.0,4.0,0.058402,2.0,2.0,1.000000,3.0,3.0,0.566290
7,BM25-Rand10-NoConFT-CBOW1,0.726637,0.188119,0.856436,0.866337,0.881188,10.0,17.0,0.359202,3.0,...,0.656968,3.0,4.0,0.707450,1.0,3.0,0.319724,3.0,5.0,0.287281
8,BM25-Rand10-NoConFT-CBOW2,0.726637,0.188119,0.856436,0.866337,0.881188,10.0,17.0,0.359202,3.0,...,0.656968,3.0,4.0,0.707450,1.0,3.0,0.319724,3.0,5.0,0.287281


In [92]:
pt.Experiment(
    [
bm25v10,
bm25v13,
bm25v14,
bm25v15,
bm25v16,
bm25v23,
bm25v24,
bm25v25,
bm25v26,
bm25v33,
bm25v34,
bm25v35,
bm25v36,
bm25v43,
bm25v44,
bm25v45,
bm25v46,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
#     names = ['BM25',
#  'BM25-Rand10-ConW2V-SG1',
#  'BM25-Rand10-ConW2V-SG2',
#  'BM25-Rand10-ConW2V-CBOW1',
#  'BM25-Rand10-ConW2V-CBOW2',
#  'BM25-Rand10-ConFT-SG1',
#  'BM25-Rand10-ConFT-SG2',
#  'BM25-Rand10-ConFT-CBOW1',
#  'BM25-Rand10-ConFT-CBOW2'],
    verbose=True,
    baseline=0
)

pt.Experiment:   0%|          | 0/17 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,"RankCutoff(BR(BM25), 30)",0.742088,0.186139,0.861386,0.876238,0.900990,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.697899,0.178218,0.831683,0.866337,0.905941,9.0,22.0,0.048380,0.0,...,0.044941,2.0,6.0,0.109176,2.0,4.0,0.416930,3.0,2.0,0.656968
2,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.704128,0.182178,0.831683,0.856436,0.900990,12.0,18.0,0.072318,0.0,...,0.158330,1.0,5.0,0.083249,1.0,4.0,0.158330,2.0,2.0,1.000000
3,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.697932,0.178218,0.831683,0.866337,0.905941,9.0,22.0,0.048543,0.0,...,0.044941,2.0,6.0,0.109176,2.0,4.0,0.416930,3.0,2.0,0.656968
4,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.704128,0.182178,0.831683,0.856436,0.900990,12.0,18.0,0.072318,0.0,...,0.158330,1.0,5.0,0.083249,1.0,4.0,0.158330,2.0,2.0,1.000000
5,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.674176,0.172277,0.831683,0.851485,0.881188,4.0,28.0,0.003184,0.0,...,0.007509,3.0,6.0,0.158330,2.0,5.0,0.166650,3.0,4.0,0.319724
6,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.689450,0.180198,0.826733,0.866337,0.881188,5.0,24.0,0.017492,0.0,...,0.083249,2.0,6.0,0.089635,1.0,2.0,0.416930,3.0,4.0,0.319724
7,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.676930,0.172277,0.841584,0.861386,0.886139,4.0,26.0,0.002323,0.0,...,0.007509,2.0,4.0,0.250179,1.0,3.0,0.258873,2.0,3.0,0.368320
8,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.688057,0.180198,0.836634,0.871287,0.891089,5.0,23.0,0.009937,0.0,...,0.083249,1.0,4.0,0.132360,0.0,1.0,0.319724,2.0,2.0,0.529762
9,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.722161,0.182178,0.841584,0.876238,0.891089,7.0,21.0,0.253924,2.0,...,0.416930,1.0,4.0,0.158330,2.0,2.0,1.000000,3.0,3.0,0.566290


#### BM25 Random 10

In [93]:
# Non-Contextualized
bm25v11 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v12 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v21 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v22 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v31 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v32 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v41 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v42 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30

# Contextualized
bm25v13 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v14 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v15 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v16 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30

bm25v23 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v24 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v25 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v26 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30

bm25v33 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v34 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v35 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v36 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30

bm25v43 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v44 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v45 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30
bm25v46 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30

In [94]:
pt.Experiment(
    [
bm25v10,
bm25v11,
bm25v12,
bm25v21,
bm25v22,
bm25v31,
bm25v32,
bm25v41,
bm25v42,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
#     names = ['BM25',
#  'BM25-Rand10-NoConW2V-SG1',
#  'BM25-Rand10-NoConW2V-SG2',
#  'BM25-Rand10-NoConW2V-CBOW1',
#  'BM25-Rand10-NoConW2V-CBOW2',
#  'BM25-Rand10-NoConFT-SG1',
#  'BM25-Rand10-NoConFT-SG2',
#  'BM25-Rand10-NoConFT-CBOW1',
#  'BM25-Rand10-NoConFT-CBOW2'],
    verbose=True,
    baseline=0
)

pt.Experiment:   0%|          | 0/9 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,"RankCutoff(BR(BM25), 30)",0.742088,0.186139,0.861386,0.876238,0.900990,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.689310,0.176238,0.826733,0.871287,0.900990,9.0,24.0,0.029052,1.0,...,0.058402,1.0,6.0,0.051732,4.0,6.0,0.819851,3.0,3.0,1.000000
2,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.689360,0.176238,0.826733,0.871287,0.905941,9.0,24.0,0.029201,1.0,...,0.058402,1.0,6.0,0.051732,4.0,6.0,0.819851,3.0,2.0,0.656968
3,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.676636,0.174257,0.821782,0.856436,0.891089,9.0,26.0,0.007293,2.0,...,0.057385,1.0,6.0,0.044941,3.0,5.0,0.287281,3.0,3.0,0.566290
4,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.687860,0.174257,0.826733,0.856436,0.881188,10.0,26.0,0.026084,1.0,...,0.033206,1.0,5.0,0.070496,1.0,4.0,0.158330,1.0,3.0,0.207485
5,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.724667,0.180198,0.836634,0.871287,0.905941,8.0,20.0,0.299482,2.0,...,0.258873,0.0,4.0,0.058402,1.0,2.0,0.566290,4.0,2.0,0.740688
6,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.719717,0.180198,0.836634,0.871287,0.905941,8.0,21.0,0.200792,2.0,...,0.258873,0.0,4.0,0.058402,1.0,2.0,0.566290,4.0,2.0,0.740688
7,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.732868,0.184158,0.846535,0.876238,0.900990,9.0,15.0,0.576347,2.0,...,0.656968,2.0,5.0,0.258873,1.0,1.0,1.000000,4.0,3.0,1.000000
8,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.732868,0.184158,0.846535,0.876238,0.900990,9.0,15.0,0.576347,2.0,...,0.656968,2.0,5.0,0.258873,1.0,1.0,1.000000,4.0,3.0,1.000000


In [95]:
pt.Experiment(
    [
bm25v10,
bm25v13,
bm25v14,
bm25v15,
bm25v16,
bm25v23,
bm25v24,
bm25v25,
bm25v26,
bm25v33,
bm25v34,
bm25v35,
bm25v36,
bm25v43,
bm25v44,
bm25v45,
bm25v46,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
#     names = ['BM25',
#  'BM25-Rand10-ConW2V-SG1',
#  'BM25-Rand10-ConW2V-SG2',
#  'BM25-Rand10-ConW2V-CBOW1',
#  'BM25-Rand10-ConW2V-CBOW2',
#  'BM25-Rand10-ConFT-SG1',
#  'BM25-Rand10-ConFT-SG2',
#  'BM25-Rand10-ConFT-CBOW1',
#  'BM25-Rand10-ConFT-CBOW2'],
    verbose=True,
    baseline=0
)

pt.Experiment:   0%|          | 0/17 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,"RankCutoff(BR(BM25), 30)",0.742088,0.186139,0.861386,0.876238,0.900990,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.704225,0.180198,0.826733,0.871287,0.900990,9.0,22.0,0.085876,1.0,...,0.181012,1.0,6.0,0.051732,4.0,6.0,0.819851,3.0,3.0,1.000000
2,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.716104,0.184158,0.836634,0.876238,0.900990,10.0,19.0,0.226498,1.0,...,0.566290,1.0,4.0,0.132360,3.0,4.0,1.000000,2.0,2.0,1.000000
3,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.704275,0.180198,0.826733,0.871287,0.905941,9.0,22.0,0.086281,1.0,...,0.181012,1.0,6.0,0.051732,4.0,6.0,0.819851,3.0,2.0,0.656968
4,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.716130,0.184158,0.836634,0.876238,0.900990,10.0,19.0,0.226957,1.0,...,0.566290,1.0,4.0,0.132360,3.0,4.0,1.000000,2.0,2.0,1.000000
5,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.686583,0.176238,0.821782,0.866337,0.891089,9.0,24.0,0.018080,2.0,...,0.095751,1.0,6.0,0.044941,4.0,5.0,0.639676,3.0,3.0,0.566290
6,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.698978,0.182178,0.821782,0.876238,0.891089,9.0,21.0,0.056358,1.0,...,0.319724,1.0,6.0,0.044941,2.0,2.0,1.000000,3.0,3.0,0.566290
7,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.697491,0.176238,0.836634,0.856436,0.881188,10.0,24.0,0.056672,1.0,...,0.058402,1.0,4.0,0.132360,1.0,4.0,0.158330,1.0,3.0,0.207485
8,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.709735,0.184158,0.836634,0.876238,0.881188,10.0,21.0,0.148706,1.0,...,0.566290,1.0,4.0,0.132360,0.0,0.0,NaN,1.0,3.0,0.207485
9,"Compose(pt.apply.query(), RankCutoff(BR(BM25),...",0.741336,0.180198,0.836634,0.876238,0.905941,9.0,17.0,0.966550,2.0,...,0.258873,0.0,4.0,0.058402,2.0,2.0,1.000000,4.0,2.0,0.740688


#### TF-IDF Rand 10

In [96]:
# Non-Contextualized
tfidfv11 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv12 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv21 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv22 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv31 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv32 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv41 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0, False)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv42 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0, False)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30

# Contextualized
tfidfv13 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv14 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv15 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv16 = pt.apply.query(lambda x: expand_query(x, w2v_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30

tfidfv23 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv24 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv25 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv26 = pt.apply.query(lambda x: expand_query(x, w2v_cbow, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30

tfidfv33 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv34 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv35 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv36 = pt.apply.query(lambda x: expand_query(x, fasttext_sg, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30

tfidfv43 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv44 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.4, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv45 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.5, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
tfidfv46 = pt.apply.query(lambda x: expand_query(x, fasttext_cb, 0.45, 0.6, True)) >> pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30

In [97]:
pt.Experiment(
    [
tfidfv10,
tfidfv13,
tfidfv14,
tfidfv15,
tfidfv16,
tfidfv23,
tfidfv24,
tfidfv25,
tfidfv26,
tfidfv33,
tfidfv34,
tfidfv35,
tfidfv36,
tfidfv43,
tfidfv44,
tfidfv45,
tfidfv46,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
#     names = ['TF-IDF',
#  'TF-IDF-Rand10-NoConW2V-SG1',
#  'TF-IDF-Rand10-NoConW2V-SG2',
#  'TF-IDF-Rand10-NoConW2V-CBOW1',
#  'TF-IDF-Rand10-NoConW2V-CBOW2',
#  'TF-IDF-Rand10-NoConFT-SG1',
#  'TF-IDF-Rand10-NoConFT-SG2',
#  'TF-IDF-Rand10-NoConFT-CBOW1',
#  'TF-IDF-Rand10-NoConFT-CBOW2'],
    verbose=True,
    baseline=0
)

pt.Experiment:   0%|          | 0/17 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,"RankCutoff(BR(TF_IDF), 30)",0.741401,0.190099,0.856436,0.876238,0.891089,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.704751,0.180198,0.826733,0.871287,0.900990,9.0,22.0,0.098305,0.0,...,0.058402,2.0,6.0,0.109176,5.0,7.0,0.828506,3.0,2.0,0.482235
2,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.716595,0.184158,0.836634,0.876238,0.900990,10.0,18.0,0.254511,0.0,...,0.181012,2.0,4.0,0.250179,4.0,5.0,1.000000,3.0,2.0,0.482235
3,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.704779,0.180198,0.826733,0.871287,0.900990,9.0,22.0,0.098566,0.0,...,0.058402,2.0,6.0,0.109176,5.0,7.0,0.828506,3.0,2.0,0.482235
4,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.716627,0.184158,0.836634,0.876238,0.900990,10.0,18.0,0.255121,0.0,...,0.181012,2.0,4.0,0.250179,4.0,5.0,1.000000,3.0,2.0,0.482235
5,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.685176,0.178218,0.826733,0.861386,0.886139,8.0,25.0,0.020771,0.0,...,0.013564,2.0,5.0,0.134334,4.0,6.0,0.494022,3.0,3.0,0.740688
6,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.698396,0.180198,0.826733,0.876238,0.891089,9.0,22.0,0.068285,0.0,...,0.024597,3.0,6.0,0.158330,3.0,3.0,1.000000,3.0,2.0,1.000000
7,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.692728,0.174257,0.821782,0.856436,0.881188,9.0,23.0,0.030440,0.0,...,0.010704,2.0,6.0,0.089635,2.0,5.0,0.207485,1.0,2.0,0.416930
8,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.704444,0.180198,0.836634,0.871287,0.881188,9.0,20.0,0.088896,0.0,...,0.058402,3.0,5.0,0.287281,1.0,2.0,0.566290,1.0,2.0,0.416930
9,"Compose(pt.apply.query(), RankCutoff(BR(TF_IDF...",0.735976,0.182178,0.826733,0.876238,0.900990,9.0,16.0,0.758269,2.0,...,0.158330,0.0,5.0,0.033206,3.0,3.0,1.000000,3.0,1.0,0.319724


In [98]:
pt.Experiment(
    [
tfidfv10,
tfidfv11,
tfidfv12,
tfidfv21,
tfidfv22,
tfidfv31,
tfidfv32,
tfidfv41,
tfidfv42,
     ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = ['TF-IDF',
 'TF-IDF-Rand10-ConW2V-SG1',
 'TF-IDF-Rand10-ConW2V-SG2',
 'TF-IDF-Rand10-ConW2V-CBOW1',
 'TF-IDF-Rand10-ConW2V-CBOW2',
 'TF-IDF-Rand10-ConFT-SG1',
 'TF-IDF-Rand10-ConFT-SG2',
 'TF-IDF-Rand10-ConFT-CBOW1',
 'TF-IDF-Rand10-ConFT-CBOW2'],
    verbose=True,
    baseline=0
)

pt.Experiment:   0%|          | 0/9 [00:00<?, ?system/s]

,name,recip_rank,P@5,R@10,R@15,R@20,recip_rank +,recip_rank -,recip_rank p-value,P@5 +,...,P@5 p-value,R@10 +,R@10 -,R@10 p-value,R@15 +,R@15 -,R@15 p-value,R@20 +,R@20 -,R@20 p-value
0,TF-IDF,0.741401,0.190099,0.856436,0.876238,0.891089,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TF-IDF-Rand10-ConW2V-SG1,0.689734,0.178218,0.821782,0.871287,0.905941,9.0,24.0,0.033812,0.0,...,0.033206,2.0,7.0,0.070496,5.0,7.0,0.828506,4.0,2.0,0.319724
2,TF-IDF-Rand10-ConW2V-SG2,0.689763,0.178218,0.821782,0.871287,0.905941,9.0,24.0,0.033911,0.0,...,0.033206,2.0,7.0,0.070496,5.0,7.0,0.828506,4.0,2.0,0.319724
3,TF-IDF-Rand10-ConW2V-CBOW1,0.674549,0.174257,0.826733,0.861386,0.886139,8.0,28.0,0.008591,0.0,...,0.004163,2.0,5.0,0.134334,4.0,6.0,0.494022,3.0,3.0,0.740688
4,TF-IDF-Rand10-ConW2V-CBOW2,0.682163,0.172277,0.811881,0.856436,0.881188,9.0,25.0,0.013071,0.0,...,0.006059,2.0,7.0,0.049027,2.0,5.0,0.207485,1.0,2.0,0.416930
5,TF-IDF-Rand10-ConFT-SG1,0.716733,0.180198,0.826733,0.871287,0.905941,8.0,19.0,0.143061,2.0,...,0.095751,0.0,5.0,0.033206,2.0,3.0,0.656968,4.0,1.0,0.181012
6,TF-IDF-Rand10-ConFT-SG2,0.711783,0.180198,0.826733,0.871287,0.905941,8.0,20.0,0.091075,2.0,...,0.095751,0.0,5.0,0.033206,2.0,3.0,0.656968,4.0,1.0,0.181012
7,TF-IDF-Rand10-ConFT-CBOW1,0.731577,0.184158,0.846535,0.881188,0.896040,12.0,15.0,0.560646,2.0,...,0.258873,1.0,3.0,0.319724,3.0,3.0,0.740688,3.0,2.0,0.656968
8,TF-IDF-Rand10-ConFT-CBOW2,0.731577,0.184158,0.846535,0.881188,0.896040,12.0,15.0,0.560646,2.0,...,0.258873,1.0,3.0,0.319724,3.0,3.0,0.740688,3.0,2.0,0.656968


#### Local + Beam 5

In [99]:
name = ["BM25", "TF-IDF", "TF-Bo1", "TF-KLD", "TF-RM3", "TF-AQE", "BM25-Bo1", "BM25-KLD", "BM25-RM3", "BM25-AQE"]
# local_model_ret = [bm25, tfidf]
qe_model = ["tfidfbo1", "tfidfkl", "tfidfrm3", "tfidfaq", "bm25bo1", "bm25kl", "bm25rm3", "bm25aq"]
isi = []
for i in qe_model:
  isi.append(i+"r5")
isi

['tfidfbo1r5',
 'tfidfklr5',
 'tfidfrm3r5',
 'tfidfaqr5',
 'bm25bo1r5',
 'bm25klr5',
 'bm25rm3r5',
 'bm25aqr5']

In [100]:
isi

['tfidfbo1r5',
 'tfidfklr5',
 'tfidfrm3r5',
 'tfidfaqr5',
 'bm25bo1r5',
 'bm25klr5',
 'bm25rm3r5',
 'bm25aqr5']

In [101]:
bo1b5 = pt.rewrite.Bo1QueryExpansion(index_ref_beam5, fb_terms=10, fb_docs=2)
klb5 = pt.rewrite.KLQueryExpansion(index_ref_beam5, fb_terms=10, fb_docs=2)
rm3b5 = pt.rewrite.RM3(index_ref_beam5)
aqb5 = pt.rewrite.AxiomaticQE(index_ref_beam5, fb_terms=10, fb_docs=2)
tfidfb5 = pt.BatchRetrieve(index_ref_beam5, wmodel = "TF_IDF") % 30
bm25b5 = pt.BatchRetrieve(index_ref_beam5, wmodel = "BM25") % 30

#Creating pipeline
bm25bo1b5 = bm25b5 >> bo1b5 >> bm25b5
bm25klb5 = bm25b5 >> klb5 >> bm25b5
bm25rm3b5 = bm25b5 >> rm3b5 >> bm25b5
bm25aqb5 = bm25b5 >> aqb5 >> bm25b5

tfidfbo1b5 = tfidfb5 >> bo1b5 >> tfidfb5
tfidfklb5 = tfidfb5 >> klb5 >> tfidfb5
tfidfrm3b5 = tfidfb5 >> rm3b5 >> tfidfb5
tfidfaqb5 = tfidfb5 >> aqb5 >> tfidfb5

In [102]:
pt.Experiment(
    [bm25,tfidf, tfidfbo1b5,
 tfidfklb5,
 tfidfrm3b5,
 tfidfaqb5,
 bm25bo1b5,
 bm25klb5,
 bm25rm3b5,
 bm25aqb5 ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = name,
    verbose = True
)

pt.Experiment:   0%|          | 0/10 [00:00<?, ?system/s]

,name,P@5,R@10,R@15,R@20,recip_rank
0,BM25,0.188119,0.861386,0.896040,0.896040,0.726728
1,TF-IDF,0.186139,0.856436,0.896040,0.896040,0.735206
2,TF-Bo1,0.182178,0.826733,0.861386,0.881188,0.737305
3,TF-KLD,0.182178,0.821782,0.861386,0.866337,0.723482
4,TF-RM3,0.184158,0.821782,0.846535,0.861386,0.674754
5,TF-AQE,0.190099,0.856436,0.876238,0.891089,0.741401
6,BM25-Bo1,0.184158,0.836634,0.866337,0.881188,0.732975
7,BM25-KLD,0.188119,0.836634,0.866337,0.881188,0.731234
8,BM25-RM3,0.186139,0.836634,0.846535,0.861386,0.671373
9,BM25-AQE,0.188119,0.861386,0.876238,0.900990,0.742088


#### Local + Beam 7

In [103]:
bo1b7 = pt.rewrite.Bo1QueryExpansion(index_ref_beam7, fb_terms=10, fb_docs=2)
klb7 = pt.rewrite.KLQueryExpansion(index_ref_beam7, fb_terms=10, fb_docs=2)
rm3b7 = pt.rewrite.RM3(index_ref_beam7)
aqb7 = pt.rewrite.AxiomaticQE(index_ref_beam7, fb_terms=10, fb_docs=2)
tfidfb7 = pt.BatchRetrieve(index_ref_beam7, wmodel = "TF_IDF") % 30
bm25b7 = pt.BatchRetrieve(index_ref_beam7, wmodel = "BM25") % 30

#Creating pipeline
bm25bo1b7 = bm25b7 >> bo1b7 >> bm25b7
bm25klb7 = bm25b7 >> klb7 >> bm25b7
bm25rm3b7 = bm25b7 >> rm3b7 >> bm25b7
bm25aqb7 = bm25b7 >> aqb7 >> bm25b7

tfidfbo1b7 = tfidfb7 >> bo1b7 >> tfidfb7
tfidfklb7 = tfidfb7 >> klb7 >> tfidfb7
tfidfrm3b7 = tfidfb7 >> rm3b7 >> tfidfb7
tfidfaqb7 = tfidfb7 >> aqb7 >> tfidfb7

In [104]:
pt.Experiment(
    [bm25,tfidf, tfidfbo1b7,
 tfidfklb7,
 tfidfrm3b7,
 tfidfaqb7,
 bm25bo1b7,
 bm25klb7,
 bm25rm3b7,
 bm25aqb7 ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = name,
    verbose = True
)

pt.Experiment:   0%|          | 0/10 [00:00<?, ?system/s]

,name,P@5,R@10,R@15,R@20,recip_rank
0,BM25,0.188119,0.861386,0.896040,0.896040,0.726728
1,TF-IDF,0.186139,0.856436,0.896040,0.896040,0.735206
2,TF-Bo1,0.182178,0.831683,0.861386,0.876238,0.726150
3,TF-KLD,0.184158,0.826733,0.856436,0.866337,0.722944
4,TF-RM3,0.182178,0.811881,0.846535,0.851485,0.673028
5,TF-AQE,0.188119,0.846535,0.876238,0.876238,0.746107
6,BM25-Bo1,0.184158,0.836634,0.871287,0.876238,0.722688
7,BM25-KLD,0.188119,0.831683,0.861386,0.881188,0.723727
8,BM25-RM3,0.188119,0.831683,0.856436,0.866337,0.670862
9,BM25-AQE,0.188119,0.846535,0.871287,0.876238,0.752959


#### Local + Beam 10

In [105]:
bo1b10 = pt.rewrite.Bo1QueryExpansion(index_ref_beam10, fb_terms=10, fb_docs=2)
klb10 = pt.rewrite.KLQueryExpansion(index_ref_beam10, fb_terms=10, fb_docs=2)
rm3b10 = pt.rewrite.RM3(index_ref_beam10)
aqb10 = pt.rewrite.AxiomaticQE(index_ref_beam10, fb_terms=10, fb_docs=2)
tfidfb10 = pt.BatchRetrieve(index_ref_beam10, wmodel = "TF_IDF") % 30
bm25b10 = pt.BatchRetrieve(index_ref_beam10, wmodel = "BM25") % 30

#Creating pipeline
bm25bo1b10 = bm25b10 >> bo1b10 >> bm25b10
bm25klb10 = bm25b10 >> klb10 >> bm25b10
bm25rm3b10 = bm25b10 >> rm3b10 >> bm25b10
bm25aqb10 = bm25b10 >> aqb10 >> bm25b10

tfidfbo1b10 = tfidfb10 >> bo1b10 >> tfidfb10
tfidfklb10 = tfidfb10 >> klb10 >> tfidfb10
tfidfrm3b10 = tfidfb10 >> rm3b10 >> tfidfb10
tfidfaqb10 = tfidfb10 >> aqb10 >> tfidfb10

In [106]:
pt.Experiment(
    [bm25,tfidf, tfidfbo1b10,
 tfidfklb10,
 tfidfrm3b10,
 tfidfaqb10,
 bm25bo1b10,
 bm25klb10,
 bm25rm3b10,
 bm25aqb10 ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = name,
    verbose = True
)

pt.Experiment:   0%|          | 0/10 [00:00<?, ?system/s]

,name,P@5,R@10,R@15,R@20,recip_rank
0,BM25,0.188119,0.861386,0.896040,0.896040,0.726728
1,TF-IDF,0.186139,0.856436,0.896040,0.896040,0.735206
2,TF-Bo1,0.184158,0.836634,0.856436,0.871287,0.736587
3,TF-KLD,0.188119,0.826733,0.856436,0.866337,0.725389
4,TF-RM3,0.184158,0.831683,0.846535,0.856436,0.682706
5,TF-AQE,0.190099,0.846535,0.881188,0.881188,0.741388
6,BM25-Bo1,0.188119,0.836634,0.876238,0.886139,0.743411
7,BM25-KLD,0.188119,0.821782,0.866337,0.886139,0.732975
8,BM25-RM3,0.186139,0.836634,0.856436,0.861386,0.692904
9,BM25-AQE,0.190099,0.851485,0.881188,0.891089,0.746341


#### Local + Rand 5

In [107]:
bo1r5 = pt.rewrite.Bo1QueryExpansion(index_ref_rand5, fb_terms=10, fb_docs=2)
klr5 = pt.rewrite.KLQueryExpansion(index_ref_rand5, fb_terms=10, fb_docs=2)
rm3r5 = pt.rewrite.RM3(index_ref_rand5)
aqr5 = pt.rewrite.AxiomaticQE(index_ref_rand5, fb_terms=10, fb_docs=2)
tfidfr5 = pt.BatchRetrieve(index_ref_rand5, wmodel = "TF_IDF") % 30
bm25r5 = pt.BatchRetrieve(index_ref_rand5, wmodel = "BM25") % 30

#Creating pipeline
bm25bo1r5 = bm25r5 >> bo1r5 >> bm25r5
bm25klr5 = bm25r5 >> klr5 >> bm25r5
bm25rm3r5 = bm25r5 >> rm3r5 >> bm25r5
bm25aqr5 = bm25r5 >> aqr5 >> bm25r5

tfidfbo1r5 = tfidfr5 >> bo1r5 >> tfidfr5
tfidfklr5 = tfidfr5 >> klr5 >> tfidfr5
tfidfrm3r5 = tfidfr5 >> rm3r5 >> tfidfr5
tfidfaqr5 = tfidfr5 >> aqr5 >> tfidfr5

In [108]:
pt.Experiment(
    [bm25,tfidf, tfidfbo1r5,
 tfidfklr5,
 tfidfrm3r5,
 tfidfaqr5,
 bm25bo1r5,
 bm25klr5,
 bm25rm3r5,
 bm25aqr5 ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = name,
    verbose = True
)

pt.Experiment:   0%|          | 0/10 [00:00<?, ?system/s]

,name,P@5,R@10,R@15,R@20,recip_rank
0,BM25,0.188119,0.861386,0.896040,0.896040,0.726728
1,TF-IDF,0.186139,0.856436,0.896040,0.896040,0.735206
2,TF-Bo1,0.184158,0.826733,0.866337,0.886139,0.733988
3,TF-KLD,0.184158,0.821782,0.846535,0.866337,0.730985
4,TF-RM3,0.178218,0.801980,0.846535,0.856436,0.665625
5,TF-AQE,0.184158,0.851485,0.876238,0.891089,0.737180
6,BM25-Bo1,0.184158,0.836634,0.876238,0.891089,0.735890
7,BM25-KLD,0.184158,0.821782,0.866337,0.886139,0.739133
8,BM25-RM3,0.182178,0.816832,0.856436,0.866337,0.649646
9,BM25-AQE,0.184158,0.851485,0.876238,0.896040,0.741055


#### Local + Rand 7

In [109]:
bo1r7 = pt.rewrite.Bo1QueryExpansion(index_ref_rand7, fb_terms=10, fb_docs=2)
klr7 = pt.rewrite.KLQueryExpansion(index_ref_rand7, fb_terms=10, fb_docs=2)
rm3r7 = pt.rewrite.RM3(index_ref_rand7)
aqr7 = pt.rewrite.AxiomaticQE(index_ref_rand7, fb_terms=10, fb_docs=2)
tfidfr7 = pt.BatchRetrieve(index_ref_rand7, wmodel = "TF_IDF") % 30
bm25r7 = pt.BatchRetrieve(index_ref_rand7, wmodel = "BM25") % 30

#Creating pipeline
bm25bo1r7 = bm25r7 >> bo1r7 >> bm25r7
bm25klr7 = bm25r7 >> klr7 >> bm25r7
bm25rm3r7 = bm25r7 >> rm3r7 >> bm25r7
bm25aqr7 = bm25r7 >> aqr7 >> bm25r7

tfidfbo1r7 = tfidfr7 >> bo1r7 >> tfidfr7
tfidfklr7 = tfidfr7 >> klr7 >> tfidfr7
tfidfrm3r7 = tfidfr7 >> rm3r7 >> tfidfr7
tfidfaqr7 = tfidfr7 >> aqr7 >> tfidfr7

In [110]:
pt.Experiment(
    [bm25,tfidf, tfidfbo1r7,
 tfidfklr7,
 tfidfrm3r7,
 tfidfaqr7,
 bm25bo1r7,
 bm25klr7,
 bm25rm3r7,
 bm25aqr7 ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = name,
    verbose = True
)

pt.Experiment:   0%|          | 0/10 [00:00<?, ?system/s]

,name,P@5,R@10,R@15,R@20,recip_rank
0,BM25,0.188119,0.861386,0.896040,0.896040,0.726728
1,TF-IDF,0.186139,0.856436,0.896040,0.896040,0.735206
2,TF-Bo1,0.186139,0.826733,0.856436,0.871287,0.741133
3,TF-KLD,0.184158,0.816832,0.836634,0.871287,0.734414
4,TF-RM3,0.186139,0.816832,0.836634,0.846535,0.659859
5,TF-AQE,0.188119,0.846535,0.876238,0.876238,0.745733
6,BM25-Bo1,0.184158,0.831683,0.861386,0.886139,0.733102
7,BM25-KLD,0.182178,0.821782,0.856436,0.881188,0.732128
8,BM25-RM3,0.188119,0.831683,0.846535,0.856436,0.655865
9,BM25-AQE,0.188119,0.831683,0.876238,0.881188,0.740421


#### Local + Rand 10

In [111]:
bo1r10 = pt.rewrite.Bo1QueryExpansion(index_ref_rand10, fb_terms=10, fb_docs=2)
klr10 = pt.rewrite.KLQueryExpansion(index_ref_rand10, fb_terms=10, fb_docs=2)
rm3r10 = pt.rewrite.RM3(index_ref_rand10)
aqr10 = pt.rewrite.AxiomaticQE(index_ref_rand10, fb_terms=10, fb_docs=2)
tfidfr10 = pt.BatchRetrieve(index_ref_rand10, wmodel = "TF_IDF") % 30
bm25r10 = pt.BatchRetrieve(index_ref_rand10, wmodel = "BM25") % 30

#Creating pipeline
bm25bo1r10 = bm25r10 >> bo1r10 >> bm25r10
bm25klr10 = bm25r10 >> klr10 >> bm25r10
bm25rm3r10 = bm25r10 >> rm3r10 >> bm25r10
bm25aqr10 = bm25r10 >> aqr10 >> bm25r10

tfidfbo1r10 = tfidfr10 >> bo1r10 >> tfidfr10
tfidfklr10 = tfidfr10 >> klr10 >> tfidfr10
tfidfrm3r10 = tfidfr10 >> rm3r10 >> tfidfr10
tfidfaqr10 = tfidfr10 >> aqr10 >> tfidfr10

In [112]:
a = pt.Experiment(
    [bm25,tfidf, tfidfbo1r10,
 tfidfklr10,
 tfidfrm3r10,
 tfidfaqr10,
 bm25bo1r10,
 bm25klr10,
 bm25rm3r10,
 bm25aqr10 ],
    topics_R04,
    qrels_q,
    eval_metrics=[P@5, R@10, R@15, R@20, "recip_rank"],
    names = name,
    verbose = True
)

pt.Experiment:   0%|          | 0/10 [00:00<?, ?system/s]

In [113]:
print(a.to_latex(index=True))

\begin{tabular}{llrrrrr}
\toprule
 & name & P@5 & R@10 & R@15 & R@20 & recip_rank \\
\midrule
0 & BM25 & 0.188119 & 0.861386 & 0.896040 & 0.896040 & 0.726728 \\
1 & TF-IDF & 0.186139 & 0.856436 & 0.896040 & 0.896040 & 0.735206 \\
2 & TF-Bo1 & 0.184158 & 0.826733 & 0.866337 & 0.886139 & 0.733988 \\
3 & TF-KLD & 0.184158 & 0.821782 & 0.846535 & 0.866337 & 0.730985 \\
4 & TF-RM3 & 0.178218 & 0.801980 & 0.846535 & 0.856436 & 0.665625 \\
5 & TF-AQE & 0.184158 & 0.851485 & 0.876238 & 0.891089 & 0.737180 \\
6 & BM25-Bo1 & 0.184158 & 0.836634 & 0.876238 & 0.891089 & 0.735890 \\
7 & BM25-KLD & 0.184158 & 0.821782 & 0.866337 & 0.886139 & 0.739133 \\
8 & BM25-RM3 & 0.182178 & 0.816832 & 0.856436 & 0.866337 & 0.649646 \\
9 & BM25-AQE & 0.184158 & 0.851485 & 0.876238 & 0.896040 & 0.741055 \\
\bottomrule
\end{tabular}

